In [ ]:
import pandas as pd
import os
import pydicom as dicom
import cv2
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import sys
import morphsnakes as ms
from skimage import morphology
from skimage.measure import label
from skimage.segmentation import active_contour
from skimage import data, io, img_as_ubyte,filters
import matplotlib as mpl
%matplotlib inline
mpl.rc('image', interpolation='none')


# SINGLE SLICE




## Get image

In [ ]:
image_path = os.path.join("123456Annotated") #diff directory than jupyter
slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(image_path)
image = sitk.ReadImage(slice_filenames)

In [ ]:
MidI = sitk.GetArrayFromImage(image[100:260, 50:200, image.GetSize()[2]//2]) 
MidI8=cv2.normalize(src=MidI, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

plt.imshow(MidI8, cmap="bone")

##Inhomogeneity correction

In [ ]:
#functions for inhomogeneity correction 
def correct_subcfat(image,mask):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output = corrector.Execute( inputImage, mask)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"
    fig, axs = plt.subplots (1, 2, figsize=(10,8))
    axs[0].imshow(image, cmap='bone')
    axs[0].set_title("Before Correction")
    axs[1].imshow(image_c, cmap='bone')
    axs[1].set_title("After Correction")
    return image_c

def correct_roi(image):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output = corrector.Execute( inputImage)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"
    fig, axs = plt.subplots (1, 2, figsize=(10,8))
    axs[0].imshow(image, cmap='bone')
    axs[0].set_title("Before Correction")
    axs[1].imshow(image_c, cmap='bone')
    axs[1].set_title("After Correction")
    return image_c

#only difference between the two is in the use of a mask for the corrector


####For Subcutaneous Fat
- for creating subcfat mask

In [ ]:
def multi_otsu_slice(image):
    motsuth=filters.threshold_multiotsu(image, classes=3)
    print(motsuth)
    print (motsuth[1])
    
    regions=np.digitize(image,bins=motsuth)
    output=img_as_ubyte(regions)

    fig, ax=plt.subplots (1,4, figsize=(10,5))
    ax[0].imshow(image, cmap="bone")
    ax[0].set_title("Original")
    ax[0].axis("off")
    ax[1].hist(image.ravel(),bins=255)

    for th in motsuth:
            ax[1].axvline(th,color="r")
    ax[2].imshow(regions,cmap="Accent")
    ax[2].set_title("Multi-Otsu Result")
    ax[2].axis ("off")
    ax[3].imshow(image>motsuth[1],cmap="bone")
    ax[3].set_title("Threshold Applied")
    ax[3].axis ("off")
    return motsuth[1] #(fat+ muscle th)


In [ ]:
#Get subfat mask to use for inhomogoneity correction
#need this mask or else shadow on top right won't be corrected and prevent proper creation of subcfat mask

def IH_mask_slice(image):
  IH_th=multi_otsu_slice(image) #apply multiotsu to get subcutaneous fat threshold
  IH_mask=label(image>IH_th)
  IH_mask= (morphology.remove_small_objects(IH_mask,min_size=800, connectivity=1)) #should we remove holes too or nah??
  IH_mask= cv2.normalize(src=IH_mask, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  ret, IH_mask = cv2.threshold(IH_mask,0,1,cv2.THRESH_BINARY) 

  plt.imshow(IH_mask)
 
  IH_mask=sitk.GetImageFromArray(IH_mask) #convert to sitk image

  return IH_mask

IH1_mask=IH_mask_slice(MidI8)


In [ ]:
#Apply inhomogeneity correction with subcfat mask on MidI8  

preMidI8c_roi=correct_subcfat(MidI8,IH1_mask)


In [ ]:
#From corrected image get another subcfat mask-generates better mask to be used for correction
IH2_mask=IH_mask_slice(preMidI8c_roi)

In [ ]:
#Apply inhomogeneity correction new (better) subcfat mask to MidI8
MidI8c_subcfat=correct_subcfat(MidI8,IH2_mask) # MidI8 and not preMidI8; MidI8 has the original bias field, applying to preMidI8c doesn't correct roi area correctly (top right too bright)

####For ROI 
- will be multiplying roimask later with corrected image below to create roi


In [ ]:
#Apply inhomogeneity correction without mask - need for proper correction of roi area (too bright if use mask to correct)
#use this pic to obtain roi later

MidI8c_roi=correct_roi(MidI8) 

##Muscle Mask

In [ ]:
#Use otsu, island removal, and morphological closing to generate subcutaneous fat mask

subcfat_th=multi_otsu_slice(MidI8c_subcfat) #get threshold for subcutaneous fat 

def subcfatmask(image):
    subcfatmask=label(image>subcfat_th)
    subcfatmask = morphology.remove_small_objects(subcfatmask,min_size=600, connectivity=1)
    subcfatmask = (morphology.remove_small_holes(subcfatmask,area_threshold=200, connectivity=1))
    subcfatmask = np.uint8(subcfatmask)
    fig, axs = plt.subplots (1, 2, figsize=(10,8))
    axs[0].imshow(image, cmap='bone')
    axs[0].set_title("Corrected")
    axs[1].imshow(subcfatmask, cmap='bone')
    axs[1].set_title("Subcfat Mask")
    return subcfatmask

subcfatmask=subcfatmask(MidI8c_subcfat)

In [ ]:
def musclemask(image):
    im_ff=image.copy()
    h, w = im_ff.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_ff, mask, (0,0), 1);
    th, premusclemask = cv2.threshold(im_ff, 0, 1, cv2.THRESH_BINARY_INV)
    kernel = np.ones((15,15),np.uint8)
    musclemask = cv2.morphologyEx(premusclemask, cv2.MORPH_CLOSE, kernel)
    fig, axs = plt.subplots (1, 3, figsize=(10,8))
    axs[0].imshow(im_ff, cmap='bone')
    axs[0].set_title(f"im_ff")
    axs[1].imshow(premusclemask, cmap='bone')
    axs[1].set_title(f"premusclemask")
    axs[2].imshow(musclemask, cmap='bone')
    axs[2].set_title(f"musclemask")
    return premusclemask,musclemask
premusclemask,musclemask=musclemask(subcfatmask)

##Bone removal + Histograms

In [ ]:
#Filter image prior to edge detection to reduce boundary heterogeneity

def gaussian_filter(x=0, y=0):

  MidI8c_roi_im=sitk.GetImageFromArray(MidI8c_roi)#convert array back to image

  gaussian_im = sitk.SmoothingRecursiveGaussian(MidI8c_roi_im, 1)
  gaussian_im = sitk.GetArrayFromImage(gaussian_im)
  gaussian_im =cv2.normalize(src=gaussian_im, dst=None, alpha=0.0, beta=10.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  gaussian=gaussian_im

  plt.imshow(gaussian, cmap='bone')
        
  return gaussian
  
gaussian_filter()
filtered=gaussian_filter()

In [ ]:
#Generate edges prior to locating marrow

kernel = np.ones((5,5),np.uint8)

edges=cv2.Canny(filtered, 15, 20) 
edges= cv2.morphologyEx(edges, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))

edges = cv2.dilate(edges,np.ones((4,5),np.uint8),iterations = 1)
edges = cv2.erode(edges,np.ones((4,4),np.uint8),iterations = 1)

fig, ax = plt.subplots(1,2, figsize=(8, 4))
ax[0].imshow(MidI8c_roi, cmap='bone')
ax[1].imshow(edges)

In [ ]:
#generate marrow mask


r=np.zeros([MidI8c_roi.shape[0], MidI8c_roi.shape[1]], dtype='uint8')
contours, hierc = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #generates list of contours and hierarchy

contour_list = [] # empty list
for contour in contours: # for each item in the contours list
    approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True) #approximates polygonal curves
    area = cv2.contourArea(contour) #determines the area inside the contour
    if ((len(approx) > 8) & (area > 30) ): 
        contour_list.append(contour) #appends to contour_list if the contour is longer and area is larger than their specified values
  
boneROI = []              
for i in range (len(contour_list)): #for each item in the contour_list (4 items were appended here)
    (x,y),radius = cv2.minEnclosingCircle(contour_list[i]) #finds a circle of the minimum area enclosing a 2d point set
    center = (int(x),int(y))
    radius = int(radius)
    #cv2.circle(MidI82,center,radius,(0,255,0),2) #draws a circle
    if radius < 10: #appends to boneROI if the radius is less than the specified number (indicating bone)
        boneROI.append(contour_list[i])  

bonemw = cv2.drawContours(r, boneROI,  -1, (1,0,0), 2) #contour parameters

# fill in bone marrow
im_floodfill = bonemw.copy()

h, w = im_floodfill.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8) # Create 0's mask 

cv2.floodFill(im_floodfill, mask, (0,0), 255);    # Floodfill from point (0, 0) with 255 value 
 
im_floodfill_inv = cv2.bitwise_not(im_floodfill)   # Invert floodfilled image
 
bonemwff = bonemw | im_floodfill_inv # Combine the two images to get the foreground.

th, bonem = cv2.threshold(bonemwff, 0, 1, cv2.THRESH_BINARY)

fig, axs = plt.subplots (1, 2, figsize=(12,4))
axs[0].imshow(bonemw,cmap="bone")
axs[1].imshow(bonem,"bone")

In [ ]:
#Superimpose marrow mask on MRI to check validity

fig, axs = plt.subplots (1, 2, figsize=(10,5))
def check(im, roi, x):
    overlay = np.ma.masked_where(roi == 0, roi)
    axs[x].imshow(im, cmap="bone")
    axs[x].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.5)
axs[0].imshow(MidI8c_roi, cmap='bone')
check(MidI8c_roi, bonem, 1)

In [ ]:
#Generate final ROI

roimask=musclemask-bonem 
plt.imshow(roimask, cmap='bone')

In [ ]:
#Comparing visuals and histograms of inhomogeneity corrected and non-corrected thigh image and ROI

roi=roimask*MidI8c_roi

fig, axs = plt.subplots (2, 4, figsize=(16,8))
axs[0,0].imshow(MidI8, cmap='bone')
axs[0,1].imshow(MidI8c_roi, cmap='bone')
axs[0,2].imshow(roimask*MidI8, cmap='bone')
axs[0,3].imshow(roi, cmap='bone')
axs[1,0].hist(MidI8.ravel(),256,[20,256]) #supressing the first 20 pixels (which represent the outer black region)
axs[1,1].hist(MidI8c_roi.ravel(),256,[20,256])
axs[1,2].hist((roimask*MidI8).ravel(),256,[1,256]) #supressing the 0 pixel (representing the outer black region)
axs[1,3].hist(roi.ravel(),256,[1,256])

plt.show

##Fat segmentation 

### Threshold Optimization #1

In [ ]:
#Get ITSA initiating threshold

initial_th1=multi_otsu_slice(roi)

In [ ]:
#Display threshold evolution and image evolution

k=1
ThPrev=0 #previous
ThRev=initial_th1 #starting point! we compare newest threshold to prev one

#apply threshold
while ThRev!=ThPrev: #while new threshold is NOT equal to prev threshold

    ThPrev=ThRev #update the previous threshold for comparison in subsequent iterations
    
    prefatmask = label(roi>ThRev)
    
    fatmask = (morphology.remove_small_objects(prefatmask,min_size=8, connectivity=1))
    
    fatmask = cv2.normalize(src=fatmask, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    ret, fatseg1_mask = cv2.threshold(fatmask,0,1,cv2.THRESH_BINARY) 
    fatseg1 = fatseg1_mask*MidI8c_roi 

    preMuscSegM=roi-fatseg1  #ROI-fat=muscle

    #masking the 0's in the image to exclude in the mean calculations
    MuscSegM=np.ma.masked_where(preMuscSegM == 0, preMuscSegM)
    FatSegM=np.ma.masked_where(fatseg1==0,fatseg1) 

    #Calculate mean signal intensities
    MuscSegI=np.mean(MuscSegM)
    FatSegI=np.mean(FatSegM)
    
    #Threshold optimization equation
    ThRev=(1+((FatSegI-MuscSegI)/FatSegI))*MuscSegI 

    fig, axs = plt.subplots (1,4, figsize=(12,4))  
    axs[0].imshow(preMuscSegM, cmap='bone', vmin=0, vmax=255) 
    axs[1].imshow(MuscSegM,cmap="bone", vmin=0, vmax=255)
    axs[2].imshow(fatseg1,cmap="bone", vmin=0, vmax=255)
    axs[3].imshow(FatSegM,cmap="bone", vmin=0, vmax=255) 
    
    print(f"Revised Threshold={ThRev}\n\tMean MUSCLE Intensity={MuscSegI} pixels\n\tMean FAT Intensity={FatSegI} pixels\n\n")

    k+=1
    if k==50:
        break
    

### Threshold Optimization #2

In [ ]:
#Remove first round of fat

roi2 = roi-fatseg1

fig, ax = plt.subplots (1, 3, figsize=(20,8))
ax[0].imshow(roi, vmin=0, vmax=255, cmap='bone') 
ax[1].imshow(fatseg1, vmin=0, vmax=255,cmap='bone')
ax[2].imshow(roi2, vmin=0, vmax=255,cmap='bone')

In [ ]:
# initial threshold for optimization loop using multi-otsu

initial_th2=multi_otsu_slice(roi2)

In [ ]:
k=1

ThPrev2=0 #previous 
ThRev2=initial_th2 #revised

while ThRev2!=ThPrev2: 

    ThPrev2=ThRev2  #update the previous threshold for comparison in subsequent iterations
    
    prefatmask2 = label(roi2>ThRev2)
    
    fatmask2 = (morphology.remove_small_objects(prefatmask2,min_size=8, connectivity=1))
    fatmask2 = cv2.normalize(src=fatmask2, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    ret, fatseg2_mask = cv2.threshold(fatmask2,0,1,cv2.THRESH_BINARY) 
    fatseg2 = fatseg2_mask*roi2

    preMuscSegM2=roi2-fatseg2  #ROI-fat=muscle

    #masking the 0's in the image to exclude in the mean calculations
    MuscSegM2=np.ma.masked_where(preMuscSegM2 == 0, preMuscSegM2)
    FatSegM2=np.ma.masked_where(fatseg2==0,fatseg2)

    #Calculate mean signal intensities
    MuscSegI2=np.mean(MuscSegM2)
    FatSegI2=np.mean(FatSegM2)

    #Threshold optimization equation   
    ThRev2=(1+((FatSegI2-MuscSegI2)/FatSegI2))*MuscSegI2 

    fig, axs = plt.subplots (1,4, figsize=(12,4))  
    axs[0].imshow(preMuscSegM2, cmap='bone', vmin=0, vmax=255) 
    axs[1].imshow(MuscSegM2,cmap="bone", vmin=0, vmax=255)#took mean intensity of this
    axs[2].imshow(fatseg2,cmap="bone", vmin=0, vmax=255) #threshold for this is based on the manual threshold (85)
    axs[3].imshow(FatSegM2,cmap="bone", vmin=0, vmax=255) #took mean intensity of this
    
    print(f"Revised Threshold={ThRev2}\n\tMean MUSCLE Intensity={MuscSegI2} pixels\n\tMean FAT Intensity={FatSegI2} pixels\n\n")
   
    k+=1
    if k==50:
        break



In [ ]:
plt.imshow(fatseg2_mask, cmap='bone')

In [ ]:
#For visualization

#combining the first and second fat segmentations
combined=fatseg1+fatseg2

#Refine fatseg2: allow object connectivity with fatseg1
fatsegfinal_mask = label(roi>ThRev2)
fatsegfinal_mask = (morphology.remove_small_objects(fatsegfinal_mask,min_size=8, connectivity=1))
fatsegfinal_mask = cv2.normalize(src=fatsegfinal_mask, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
ret, fatsegfinal_mask = cv2.threshold(fatsegfinal_mask,0,1,cv2.THRESH_BINARY) 
fatsegfinal = fatsegfinal_mask*roi               

fatseg2_refined_mask = (fatsegfinal != fatseg1).astype(int) #Isolate refined second fat segmentation
fatseg2_refined=fatseg2_refined_mask*roi

fig, axs = plt.subplots (1,5, figsize=(12,4))  
axs[0].imshow(fatseg1, cmap='bone', vmin=0, vmax=255) 
axs[0].set_title("fatseg1")
axs[1].imshow(fatseg2,cmap="bone", vmin=0, vmax=255)
axs[1].set_title("fatseg2")
axs[2].imshow(fatseg2_refined,cmap="bone", vmin=0, vmax=255)
axs[2].set_title("fatseg2 refined")
axs[3].imshow(combined,cmap="bone", vmin=0, vmax=255)
axs[3].set_title("fatseg1+fatseg2")
axs[4].imshow(fatsegfinal,cmap="bone", vmin=0, vmax=255) #XY connectivity between fatseg1 and fatseg2 considered in island removal
axs[4].set_title("refined")

### Final Product


In [ ]:
def check2(im, roi,x,y): 
    overlay = np.ma.masked_where(roi == 0, roi)
    axs[x,y].imshow(im, cmap="bone")
    axs[x,y].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.6)

def check3(im, fat1, fat2,x,y):  
    overlay1 = np.ma.masked_where(fat1 == 0, fat1)
    overlay2 = np.ma.masked_where(fat2 == 0, fat2)

    axs[x,y].imshow(im, cmap="bone")
    axs[x,y].imshow(overlay2, cmap="bwr", vmin=0, vmax=1)
    axs[x,y].imshow(overlay1, cmap="autumn", vmin=0, vmax=1 )

fig, axs = plt.subplots (3, 3, figsize=(12,12))

check3(fatseg1vol_mask_R3_ken[7], fatseg1vol_mask_R3_ken[7], fatseg2_refinedvol_mask_R3_ken[7],2,1) 


## Fat/Muscle Data

### Calculations

#####Threshold 1 + 2 Separate

In [ ]:
#run calculations for Th1 + Th2 separately (for checking with bigger function below)

from tabulate import tabulate #for table multiple headers
def calc_slice(fat):
  im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
  muscleroi=roi-fat 
  #MUSCLE AND FAT AREA
  MuscFatAreaPix=np.sum(roi>0) 
  MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1] #multiply by x and y to convert to mm:  1 pixel = spacing 0.9765625,0.9765625,5.0
  #FAT AREA
  FatAreaPix=np.sum(fat>0) 
  FatAreaMM=FatAreaPix*im_spacing[0]*im_spacing[1]
  #MUSCLE AREA
  MuscAreaPix=np.sum(muscleroi>0)
  MuscAreaMM=MuscAreaPix*im_spacing[0]*im_spacing[1]
  #FAT PERCENTAGE
  FatPerc=FatAreaPix*100/MuscFatAreaPix #adding 2 dashes rounds it DOWN to whole number
  #VOLUME OF EACH SLICE
  MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
  FatVolMM=FatAreaMM*im_spacing[2] #multiply by z -slice thickness
  MuscVolMM=MuscAreaMM*im_spacing[2]
  print(f"\nSlice 8 Threshold #1 \n")
  print(tabulate([['Musc + Fat Area 1', "%.1f" % MuscFatAreaPix,'pixels'], ['Musc + Fat Area 1',"%.1f" % MuscFatAreaMM,'mm^2'],['Musc + Fat Vol 1', "%.1f" % MuscFatVolMM,'mm^3'],[],
          ['Musc Area 1',"%.1f" % MuscAreaPix,'pixels'],['Musc Area 1',"%.1f" % MuscAreaMM,'mm^2'],['Musc Vol 1', "%.1f" % MuscVolMM,'mm^3'],[],
          ['Fat Area 1', "%.1f" % FatAreaPix,'pixels'],  ['Fat Area 1', "%.1f" % FatAreaMM,'mm^2'], 
          ['Fat Perc 1', "%.1f" % FatPerc,'%'],['Fat Vol', "%.1f" % FatVolMM,'mm^3']],
          headers=['Data', 'Value','Units']))
  print (f"\n")
#calc_slice(fatseg1) #threshold 1 fat
calc_slice(fatsegfinal) #threshold 2 fat

#####Threshold 1 + 2 Combined + Fat Correction

In [ ]:
def calc_slice(fat1,fat2,fat3):
    im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
   
  #MUSCLE + FAT AREA
    MuscFatAreaPix=np.sum(roi>0)   #not roi2vol cus has no fat there! - don't need this line for 2nd round (taken care of in first round) #same result as roimask>0 #not musclemask>0 cus musclemask INCLUDES bone area! roi is ONLY muscle+fat. #Also don’t just do roi, need roi>0
    MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1] #multiply by x and y to convert to mm:  1 pixel = spacing 0.9765625,0.9765625,5.0
    MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
  
  #THRESHOLD 1 FAT
    muscleroi1=roi-fat1 #muscleroi=preMuscSegM from loop
 
    #FAT AREA
    FatAreaPix1=np.sum(fat1>0) 
    FatAreaMM1=FatAreaPix1*im_spacing[0]*im_spacing[1]
    
    #MUSCLE AREA
    MuscAreaPix1=np.sum(muscleroi1>0)
    MuscAreaMM1=MuscAreaPix1*im_spacing[0]*im_spacing[1]
    
    #FAT PERCENTAGE
    FatPerc1=FatAreaPix1*100/MuscFatAreaPix #adding 2 dashes rounds it DOWN to whole number
    
    #VOLUME OF EACH SLICE
    FatVolMM1=FatAreaMM1*im_spacing[2] #multiply by z -slice thickness
    MuscVolMM1=MuscAreaMM1*im_spacing[2]
    print(f"\nThreshold #1 \n")

    print(tabulate([['Musc + Fat Area', "%.1f" % MuscFatAreaPix,'pixels'], ['Musc + Fat Area',"%.1f" % MuscFatAreaMM,'mm^2'],['Musc + Fat Vol', "%.1f" % MuscFatVolMM,'mm^3'],[],
            ['Musc Area 1',"%.1f" % MuscAreaPix1,'pixels'],['Musc Area 1',"%.1f" % MuscAreaMM1,'mm^2'],['Musc Vol 1', "%.1f" % MuscVolMM1,'mm^3'],[],
            ['Fat Area 1', "%.1f" % FatAreaPix1,'pixels'],  ['Fat Area 1', "%.1f" % FatAreaMM1,'mm^2'], 
            ['Fat Perc 1', "%.1f" % FatPerc1,'%'],['Fat Vol 1', "%.1f" % FatVolMM1,'mm^3']],
            headers=['Data', 'Value','Units']))
    print (f"\n")
        
  #THRESHOLD 2 FAT
    #NOT including threshold 1 fat----------------------------------------------
    muscleroi2=roi-fat2 
    
    #FAT AREA
    FatAreaPix2=np.sum(fat2>0)
    FatAreaMM2=FatAreaPix2*im_spacing[0]*im_spacing[1]
    
    #MUSCLE AREA
    MuscAreaPix2=np.sum(muscleroi2>0)
    MuscAreaMM2=MuscAreaPix2*im_spacing[0]*im_spacing[1]
    
    #FAT PERCENTAGE
    FatPerc2=FatAreaPix2*100/MuscFatAreaPix
    
    #VOLUME OF EACH SLICE
    FatVolMM2=FatAreaMM2*im_spacing[2]
    MuscVolMM2=MuscAreaMM2*im_spacing[2]
  
    #FAT CORRECTION
    Fat1masked=np.ma.masked_where(fat1==0,fat1) #brighter fat
    Fat2masked=np.ma.masked_where(fat2==0,fat2) #less bright fat
    FatSegI_1=np.mean(Fat1masked)
    FatSegI_2=np.mean(Fat2masked)
    cfactor=(FatSegI_2/FatSegI_1) #fat correction factor
    
    FatVolCombined=FatVolMM1+FatVolMM2 #final volume not corrected
    FatVolMM2_C=FatVolMM2*cfactor 
    FatVolCombined_C=FatVolMM1+FatVolMM2_C #final volume corrected
    FatPerc2_C=FatPerc2*cfactor
    
    print(f"\nThreshold #2 NOT including fat 1  \n")
    print(tabulate([['Musc Area 2',"%.1f" % MuscAreaPix2,'pixels'],['Musc Area 2',"%.1f" % MuscAreaMM2,'mm^2'],['Musc Vol 2', "%.1f" % MuscVolMM2,'mm^3'],[],
            ['Fat Area 2', "%.1f" % FatAreaPix2,'pixels'],['Fat Area 2 ', "%.1f" % FatAreaMM2,'mm^2'],
            ['Fat Perc 2', "%.1f" % FatPerc2,'%'],['Fat Perc 2 Corrected', FatPerc2_C,'%'],['Fat Vol 2 Not Corrected', "%.1f" % FatVolMM2,'mm^3'],['Fat Vol 2 Corrected', "%.1f" % FatVolMM2_C,'mm^3']],
            headers=['Data', 'Value','Units']))
    print (f"\n")
    print(f"\nFat Correction \n")
    print(tabulate([['Fat 1 Intensity', FatSegI_1,'pixels'],['Fat 2 Intensity', FatSegI_2,'mm^2'],['Fat Correction Factor', cfactor],[],],
            headers=['Data', 'Value','Units']))
    print (f"\n")

    #INCLUDES threshold 1 fat----------------------------------------------------
    muscleroi3=roi-fat3
    
    #FAT AREA
    FatAreaPix3=np.sum(fat3>0)
    FatAreaMM3=FatAreaPix3*im_spacing[0]*im_spacing[1]
    
    #MUSCLE AREA
    MuscAreaPix3=np.sum(muscleroi3>0)
    MuscAreaMM3=MuscAreaPix3*im_spacing[0]*im_spacing[1]
        
    #FAT PERCENTAGE
    FatPerc3=FatAreaPix3*100/MuscFatAreaPix
    
    #VOLUME OF EACH SLICE
    FatVolMM3=FatAreaMM3*im_spacing[2] #this is same as FatVolCombined NOT corrected, use this to confirm same volume
    MuscVolMM3=MuscAreaMM3*im_spacing[2]    
    print(f"\nThreshold #2 INCLUDING fat 1 \n")
    print(tabulate([['Musc Area 3',"%.1f" % MuscAreaPix3,'pixels'],['Musc Area 3',"%.1f" % MuscAreaMM3,'mm^2'],['Musc Vol 3', "%.1f" % MuscVolMM3,'mm^3'],[],
            ['Fat Area 3', "%.1f" % FatAreaPix3,'pixels'],['Fat Area 3 ', "%.1f" % FatAreaMM3,'mm^2'],
            ['Fat Perc 3', "%.1f" % FatPerc3,'%'],['Fat Vol 3', "%.1f" % FatVolMM3,'mm^3'],[],
            ['Fat Volume Combined Not Corrected', "%.1f" % FatVolCombined,'MM^3'],['Fat Volume Combined Corrected', "%.1f" % FatVolCombined_C,'MM^3']],
            headers=['Data', 'Value','Units']))
    print ("------------------------------------------------------------")
   
calc_slice(fatseg1,fatseg2_refined,fatsegfinal)


### Summary tables + Graphs

# VOLUME

##Get Images

In [ ]:
Ivol=sitk.GetArrayFromImage(image[100:260, 50:200, :]) #get all slices

Ivol8= np.zeros([Ivol.shape[0], Ivol.shape[1], Ivol.shape[2]], dtype='uint8') 
for i in range(Ivol.shape[0]):
  Ivol8[i]=cv2.normalize(src=Ivol[i], dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #!note if put x[i] then when calling function put Ivol8t not Ivol8t[i]

In [ ]:
j=Ivol8.shape[0]
plotx=5

def stackimages(image, x=0, y=0):
    fig, axs = plt.subplots (j//plotx, plotx, figsize=(17,10))
    for i in range(j):
        axs[y, x].imshow(image[i], cmap='bone')
        axs[y, x].set_title(f"slice {i+1}", fontsize=12)
        if x <(plotx-1):
            x+=1
        else:
            x=0
            y+=1

### Inhomogeneity Correction

####For Subcutaneous Fat

In [ ]:
def multi_otsu(image): #only difference from multi_otsu_slice is title (includes slice # here)
    motsuth=filters.threshold_multiotsu(image, classes=3)
    print(motsuth)
    print (f"Slice {i+1} otsu threshold={motsuth[1]}")
    
    regions=np.digitize(image,bins=motsuth)
    output=img_as_ubyte(regions)

    fig, ax=plt.subplots (1,4, figsize=(10,5))
    ax[0].imshow(image, cmap="bone")
    ax[0].set_title(f"Slice {i+1} Original")
    ax[0].axis("off")
    ax[1].hist(image.ravel(),bins=255)

    for th in motsuth:
            ax[1].axvline(th,color="r")
    ax[2].imshow(regions,cmap="Accent")
    ax[2].set_title("Multi-Otsu Result")
    ax[2].axis ("off")
    ax[3].imshow(image>motsuth[1],cmap="bone")
    ax[3].set_title("Threshold Applied")
    ax[3].axis ("off")
    return motsuth[1] #(fat+ muscle th)

In [ ]:
def IH_mask(image):
  IH_th=multi_otsu(image) #apply multiotsu to get subcutaneous fat threshold
  IH_mask=label(image>IH_th)
  IH_mask= (morphology.remove_small_objects(IH_mask,min_size=800, connectivity=1)) #should we remove holes too or nah??
  IH_mask= cv2.normalize(src=IH_mask, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  ret, IH_mask = cv2.threshold(IH_mask,0,1,cv2.THRESH_BINARY) 

  plt.imshow(IH_mask)
 
  IH_mask=sitk.GetImageFromArray(IH_mask) #convert to sitk image

  return IH_mask


In [ ]:
stackimages(Ivol8)

In [ ]:
#Get subfat mask to use for inhomogoneity correction
#need this mask or else shadow on top right won't be corrected and prevent proper creation of subcfat mask

IH1vol_mask=[] 
for i in range(j):
  IH1vol_mask.append([])
  IH1vol_mask[i]=IH_mask(Ivol8[i])
 

In [ ]:
#convert from image to array to view masks
show_IH1vol_mask= np.zeros([Ivol.shape[0], Ivol.shape[1], Ivol.shape[2]], dtype='uint8')

for i in range(j):
  show_IH1vol_mask[i]=sitk.GetArrayFromImage(IH1vol_mask[i])

stackimages(show_IH1vol_mask)

In [ ]:
#Apply inhomogeneity correction with subcfat masks on Ivol8 slices

preIvol8c_roi= np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

for i in range(j):
  preIvol8c_roi[i]=correct_subcfat(Ivol8[i],IH1vol_mask[i])

In [ ]:
#From corrected images get another subcfat mask for each slice - generates better masks to be used for correction

IH2vol_mask=[]
for i in range(j):
  IH2vol_mask.append([])
  IH2vol_mask[i]=IH_mask(preIvol8c_roi[i])

In [ ]:
#Apply inhomogeneity correction new (better) subcfat mask to Ivol8c slices
Ivol8c_subcfat= np.zeros([Ivol.shape[0], Ivol.shape[1], Ivol.shape[2]], dtype='uint8')

for i in range(j):  
  Ivol8c_subcfat[i]=correct_subcfat(Ivol8[i],IH2vol_mask[i])

####For ROI

In [ ]:
#Apply inhomogeneity correction without mask - need for proper correction of roi area (too bright if use mask to correct)
#use these images to obtain roi later

Ivol8c_roi= np.zeros([Ivol.shape[0], Ivol.shape[1], Ivol.shape[2]], dtype='uint8')

for i in range(j):
  Ivol8c_roi[i]=correct_roi(Ivol8[i])

### Histograms - Uncorrected vs. Corrected images

In [ ]:
fig, axs = plt.subplots (j, 4, figsize=(17,63.75))

def hist(image):

    for i in range(j):
        axs[i, 0].set_title(f"slice {i+1}", fontsize=12)
        axs[i, 0].imshow(Ivol8[i], cmap='bone')
        axs[i, 1].hist(Ivol8.ravel(),256,[17,256]) #pixels<17 suppressed
        axs[i, 2].imshow(image[i], cmap='bone')
        axs[i, 3].hist(image[i].ravel(),256,[17,256]) #pixels<17 suppressed

hist(Ivol8c_roi)

##Muscle Mask

In [ ]:
def subcfat(image, x=0, y=0):
    
    fatotsvol=subcfatvol=presubcfatvol=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    subcfatvol_th=[]

    for i in range(j):
        subcfatvol_th.append([])
        subcfatvol_th[i]=multi_otsu(image[i])

        otsuth, fatotsvol[i] = cv2.threshold(image[i],subcfatvol_th[i],255,cv2.THRESH_BINARY)
        presubcfatvol[i] = label(fatotsvol[i])
        subcfatvol[i] = morphology.remove_small_objects(presubcfatvol[i],min_size=600, connectivity=1)
        subcfatvol[i] = (morphology.remove_small_holes(subcfatvol[i],area_threshold=16, connectivity=1)).astype(int) #or min_size (gets warning)
        subcfatvol[i]=cv2.normalize(src=subcfatvol[i], dst=None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            
        if i==(j-1):
            return subcfatvol
            
subcfatvol=subcfat(Ivol8c_subcfat)
stackimages(subcfatvol)

In [ ]:
kernel = np.ones((3,3),np.uint8)
subcfat2vol=subcfat3vol=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

for i in range(j):
    subcfat2vol[i]= cv2.dilate(subcfatvol[i],kernel,iterations = 2)
    subcfat2vol[i] = cv2.erode(subcfat2vol[i],kernel,iterations = 2)

stackimages(subcfat2vol)

In [ ]:
def musclemask(x=0, y=0):
    
    premusclemaskvol=musclemaskvol=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    im_floodfillvol=subcfat2vol.copy()
    
    for i in range(j):      
        h, w = subcfatvol[i].shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(im_floodfillvol[i], mask, (0,0), 1);
        
        th, premusclemaskvol[i] = cv2.threshold(im_floodfillvol[i], 0, 1, cv2.THRESH_BINARY_INV)
        kernel = np.ones((15,15),np.uint8)
        
        musclemaskvol[i] = cv2.morphologyEx(premusclemaskvol[i], cv2.MORPH_CLOSE, kernel)
            
        if i==j-1:
            return musclemaskvol
            
musclemaskvol = musclemask()
stackimages(musclemaskvol)

###Overall

##Refine Muscle Mask: Snakes


In [ ]:
contoursvol=[]
hiercvol=[]
contour_listvol=[]
maskoutlinevol=[]
contoursvol=[]
r=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
contour_listvol=[]
maskoutlinevol=[]
contoursvol=[]
for i in range(j):
    contoursvol.append([])

    contour_listvol.append([])
    maskoutlinevol.append([])
    
for i in range (j):
    a, b =  cv2.findContours(musclemaskvol[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contoursvol[i].append(a)

    
    for contour in contoursvol[i][0]:      
        contour_listvol[i].append(contour)

    maskoutlinevol[i] = cv2.drawContours(r[i], contour_listvol[i],  -1, (1,0,0), 1)
    
stackimages(maskoutlinevol)

In [ ]:
hullvol=[]
drawingvol = np.zeros((musclemaskvol.shape[0], musclemaskvol.shape[1], musclemaskvol.shape[2], 3), np.uint8)
drawing2vol = np.zeros((musclemaskvol.shape[0], musclemaskvol.shape[1], musclemaskvol.shape[2]), np.uint8)
for i in range(j):
    hullvol.append([])  
    
    for i2 in range(len(contoursvol[i])):
        hullvol[i].append(cv2.convexHull(contoursvol[i][i2][0], False))

    for i2 in range(len(contoursvol[i])):
        color_contours = (0, 255, 0) # green - color for contours
        color_hull = (255, 0, 0) # red - color for convex hull
        cv2.drawContours(drawingvol[i], contoursvol[i][0], i2, color_contours, 1, 8)
        cv2.drawContours(drawingvol[i], hullvol[i], i2, color_hull, 1, 8)

    
    for i2 in range(len(contours)):
        color_hull = (255, 0, 0) # blue - color for convex hull
        cv2.drawContours(drawing2vol[i], hullvol[i],  -1, (1,0,0), 1)

fig, axs = plt.subplots(j, 3, figsize= (18, 64))
for i in range(j):
    axs[i, 0].set_title(f"slice {i+1}", fontsize=18)
    axs[i, 0].imshow(maskoutlinevol[i])
    axs[i, 1].imshow(drawingvol[i])
    axs[i, 2].imshow(drawing2vol[i])

In [ ]:
#generate full contour list
fullcontoursvol=[]

drawing3vol = np.zeros((musclemaskvol.shape[0], musclemaskvol.shape[1], musclemaskvol.shape[2]), np.uint8)



In [ ]:
hull2vol=[]

for i in range(j):
    hull2vol.append([])
    for i2 in range(len(fullcontoursvol[i][0][0])):
        hull2vol[i].append(fullcontoursvol[i][0][0][i2][0])
        

    hull2vol[i]=np.array(hull2vol[i])
    hull2vol[i]=hull2vol[i].astype(float)

In [ ]:
snakevol=[]

fig, axs = plt.subplots(j, 3, figsize=(18,64))

for i in range(j):
    snakevol.append([])
    snakevol[i]= active_contour(Ivol8c_roi[i], hull2vol[i],alpha=0.005, beta=8, w_line=-0.1, w_edge=1.4,gamma=0.15, convergence=0.01)
    
    axs[i, 0].set_title(f"slice {i+1}", fontsize=14)
    axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')

    axs[i,1].imshow(Ivol8c_roi[i], cmap='bone')
    axs[i,1].plot(hull2vol[i][:, 0], hull2vol[i][:, 1], '--r', lw=1)

    axs[i,2].imshow(Ivol8c_roi[i], cmap='bone')
    axs[i,2].plot(snakevol[i][:, 0], snakevol[i][:, 1], '-b', lw=1);


In [ ]:
snakevol2=[]

fig, axs = plt.subplots(j, 3, figsize=(18,64))

for i in range(j):
    snakevol2.append([])
    snakevol2[i]= active_contour(Ivol8c_roi[i], snakevol[i],alpha=0.1, beta=1, w_line=-0.6, w_edge=1,gamma=0.5, convergence=0.8)
    
    axs[i, 0].set_title(f"slice {i+1}", fontsize=14)
    axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')

    axs[i,1].imshow(Ivol8c_roi[i], cmap='bone')
    axs[i,1].plot(snakevol[i][:, 0], snakevol[i][:, 1], '-r', lw=1)

    axs[i,2].imshow(Ivol8c_roi[i], cmap='bone')
    axs[i,2].plot(snakevol2[i][:, 0], snakevol2[i][:, 1], '-b', lw=1);


In [ ]:
drawing3vol = np.zeros((musclemaskvol.shape[0], musclemaskvol.shape[1], musclemaskvol.shape[2]), np.uint8)
vol=[]

for i in range(j):
    vol.append([])
    for i2 in range(len(snakevol2[i])):
        vol[i].append([])
        vol[i][i2].append(snakevol2[i][i2])
    
    vol[i]=np.rint(vol[i]).astype(int)

    cv2.drawContours(drawing3vol[i], vol[i],  -1, (1,0,0), 1)   

    
drawsnakevol= np.zeros((musclemaskvol.shape[0], musclemaskvol.shape[1], musclemaskvol.shape[2]), np.uint8)

for i in range(j):
    cv2.polylines(drawsnakevol[i], [vol[i]], isClosed=True, color = (255, 0, 0) , thickness=1) 


stackimages(drawing3vol)
stackimages(drawsnakevol)

In [ ]:
snakemaskvol=drawsnakevol.copy()

h, w = subcfatvol[i].shape[:2]

for i in range (j):
    mask = np.zeros((h+2, w+2), np.uint8)
    (x,y),radius = cv2.minEnclosingCircle(vol[i]) #(x, y) coordinates of the centre of the "circle"
    cv2.floodFill(snakemaskvol[i], mask, (round(x),round(y)), 255)
    

ret, snakemaskvol = cv2.threshold(snakemaskvol,0,1,cv2.THRESH_BINARY)
stackimages(snakemaskvol)

In [ ]:
#ADDING SNAKE ROI AND MUSCLE ROI

musclemaskvol2=snakemaskvol+musclemaskvol
ret, musclemaskvol2 = cv2.threshold(musclemaskvol2,0,1,cv2.THRESH_BINARY)

fig, axs = plt.subplots (j, 4, figsize=(15,60))
for i in range(j):

    def check2(im, roi):    
        overlay = np.ma.masked_where(roi[i] == 0, roi[i])
        axs[i,1].imshow(im[i], cmap="bone")
        axs[i,1].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
    def check3(im, roi):    
        overlay = np.ma.masked_where(roi[i] == 0, roi[i])
        axs[i,2].imshow(im[i], cmap="bone")
        axs[i,2].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
    def check4(im, roi):    
        overlay = np.ma.masked_where(roi[i] == 0, roi[i])
        axs[i,3].imshow(im[i], cmap="bone")
        axs[i,3].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)

        
    axs[i, 0].set_title(f"slice {i+1}", fontsize=12)   
    axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')
    check2(Ivol8c_roi, musclemaskvol)
    check3(Ivol8c_roi, snakemaskvol)
    check4(Ivol8c_roi, musclemaskvol2)

In [ ]:
#island removal from above to account for overshooting... 

#s_dif_refined = np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
#for i in range(j):
#    s_dif_refined[i]=label(s_dif[i])
#    s_dif_refined[i] = morphology.remove_small_objects(s_dif_refined[i],min_size=2, connectivity=1)
#    ret, s_dif_refined[i] = cv2.threshold(s_dif_refined[i],0,1,cv2.THRESH_BINARY)

#stackimages(s_dif_refined)

In [ ]:
h, w = subcfatvol[i].shape[:2]

t=t_final=t2=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range (j):

    t[i]=cv2.Canny(musclemaskvol2[i], 1, 2) 
    ret, t[i] = cv2.threshold(t[i],0,1,cv2.THRESH_BINARY)
    t[i]=cv2.normalize(src=t[i], dst=None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    t_fill=t[i].copy()

    t_coord= np.argwhere(t[i] ==1)
    mask = np.zeros((h+2, w+2), np.uint8)
    (x,y),radius = cv2.minEnclosingCircle(t_coord)
    cv2.floodFill(t_fill, mask, (round(x),round(y)), 2)

    ret, t_final[i] = cv2.threshold(t_fill,1,1,cv2.THRESH_BINARY)
    


In [ ]:
t2=(t_final!=musclemaskvol2).astype(int)
t2=np.uint8(t2)
stackimages(t2)

In [ ]:
g=musclemaskvol2-t2
stackimages(g)

In [ ]:
erodesnake=musclemaskvol2-t2

print(t2.dtype)
print(musclemaskvol2.dtype)
stackimages(erodesnake)

#stackimages(musclemaskvol2)

In [ ]:
s_diff=musclemaskvol-erodesnake
th, s_diff = cv2.threshold(s_diff, 1, 1, cv2.THRESH_BINARY)

print(np.amax(s_diff))

stackimages(s_diff)



In [ ]:
fig, axs = plt.subplots (j, 1, figsize=(15,120))
for i in range(j):

    def check2(im, roi):    
        overlay = np.ma.masked_where(roi[i] == 0, roi[i])
        axs[i].imshow(im[i], cmap="bone")
        axs[i].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4) 
        
    axs[i].set_title(f"slice {i+1}", fontsize=12)   
    check2(Ivol8c_roi, s_diff)
    

##Bone removal

Gaussian Filter

In [ ]:
gaussianvol =np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

def gaussian_filter(x=0, y=0):
  for i in range(j):

      gaussian_im = sitk.SmoothingRecursiveGaussian(image[100:260, 50:200, i], 1)
      gaussian_im = sitk.GetArrayFromImage(gaussian_im)
      gaussian_im =cv2.normalize(src=gaussian_im, dst=None, alpha=0.0, beta=10.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
      gaussianvol[i]=gaussian_im
      
      if i==14:
          return gaussianvol  #change return to processed image
          
gaussianvol=gaussian_filter()
stackimages(gaussianvol)

In [ ]:
fig, axs = plt.subplots (j//plotx, plotx, figsize=(17,10))

def stackedges(image, x=0, y=0):
    
    edges=edges2=dilation=erosion=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    for i in range(j):
        
        kernel = np.ones((5,5),np.uint8)
        
        edges[i]=cv2.Canny(image[i], 15, 20) #Changed this
        edges[i]= cv2.morphologyEx(edges[i], cv2.MORPH_CLOSE, kernel)
        
        edges[i] = cv2.dilate(edges[i],kernel,iterations = 1)
        edges[i] = cv2.erode(edges[i],kernel,iterations = 1)
        
        axs[y, x].imshow(edges[i], vmin=0, vmax=10)
        axs[y, x].set_title(f"slice {i+1}", fontsize=12)
        
        if x <(plotx-1):            
            x+=1
        
        else:          
            x=0
            y+=1
        
        if i==14:
            return edges2  #change return to processed image
            
stackedges(gaussianvol)
edgesvol=stackedges(gaussianvol)

In [ ]:
contoursvol=[]
hiercvol=[]
contour_listvol=[]
boneROIvol=[]
bonemwvol=[]

r=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    contoursvol.append([])
    hiercvol.append([])
    contour_listvol.append([])
    boneROIvol.append([])
    bonemwvol.append([])
    
for i in range (j):
    a, b =  cv2.findContours(edgesvol[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contoursvol[i].append(a)
    hiercvol[i].append(b)
    
    for contour in contoursvol[i][0]:
        approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
        area = cv2.contourArea(contour) #determines the area inside the contour
        if ((len(approx) > 5) & (area > 100) ):  #Changed Here
            contour_listvol[i].append(contour)

    for i2 in range (len(contour_listvol[i])): 
        (x,y),radius = cv2.minEnclosingCircle(contour_listvol[i][i2]) #finds a circle of the minimum area enclosing a 2d point set
        center = (int(x),int(y))
        radius = int(radius)
        #cv2.circle(MidI82,center,radius,(0,255,0),2) #draws a circle            
        if radius < 10: #appends to boneROI if the radius is less than the specified number (indicating bone)
            boneROIvol[i].append(contour_listvol[i][i2])

    bonemwvol[i] = cv2.drawContours(r[i], boneROIvol[i],  -1, (1,0,0), 2)
    
stackimages(bonemwvol)

In [ ]:
bonemwffvol=bonemvol=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    im_floodfill = bonemwvol[i].copy()
    h, w = im_floodfill.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0,0), 255);
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    bonemwffvol[i] = bonemwvol[i] | im_floodfill_inv
    th, bonemvol[i] = cv2.threshold(bonemwffvol[i], 0, 1, cv2.THRESH_BINARY) 
        
stackimages(bonemvol)   



In [ ]:
#musclemaskvol=no snake
#musclemaskvol2= with snake

roimaskvol=erodesnake-bonemvol
              
stackimages(roimaskvol)

In [ ]:
roivol=roimaskvol*Ivol8c_roi
       
stackimages(roivol)

##Round 1: Fat segmentation

###Threshold optimization #1

In [ ]:
#initial threshold for optimization loop using multi-otsu

initial_th1_R1=[]

for i in range(j):
  initial_th1_R1.append([])
  initial_th1_R1[i]=multi_otsu(roivol[i])

In [ ]:
from astropy.table import QTable
from tabulate import tabulate

thresholds=[] 
def optimize_threshold(roi, ots): #not (fatth,roi) cus fatth needs to change with every slice
  
    k=1
    ThPrev=0
    ThRev=ots  #starting point! we compare newest threshold to prev one

    #New lists
    ThPrevlist=[ThPrev]  #include starting ThPrev - for table later
    ThRevlist=[ThRev] #include starting ThRev  - for table and graph later

    klist=[0] #iteratios - include 0th iteration - for table and graph later
    matchlist=[] #ThRev=ThPrev yes or no - for table later
    while ThRev!=ThPrev:  #while new threshold is NOT equal to prev threshold
        
        ThPrev=ThRev#make new threshold=prev threshold, new new threshold will be created at the end of loop
        
        prefatmask = label(roi>ThRev) 
        prefatmask = np.uint8(prefatmask) 
        prefatmask = (morphology.remove_small_objects(prefatmask,min_size=8, connectivity=1))
        ret, fatmask = cv2.threshold(prefatmask,0,1,cv2.THRESH_BINARY)

        fatseg = fatmask*roi #roi=roivol[i] later
        preMuscSegM=roi-fatseg #muscle-fat=muscle
        MuscSegM=np.ma.masked_where(preMuscSegM == 0, preMuscSegM)#muscle peeled
        FatSegM=np.ma.masked_where(fatseg==0,fatseg) #fat peeled
        MuscSegI=np.mean(MuscSegM) #mean signal intensity
        FatSegI=np.mean(FatSegM)
        ThRev=(1+((FatSegI-MuscSegI)/FatSegI))*MuscSegI #NOT MUSCLE-FAT
        
        ThPrevlist.append(ThPrev) 
        ThRevlist.append(ThRev) #append ThRev, starting from first iteration
        klist.append(k) #append k value, starting k=1 (first iteration)
        matchlist.append("No") 
        k+=1
        if k==50:
            break
       
        if ThRev==ThPrev:
          matchlist.append("Yes")
          table=QTable([klist,ThPrevlist,ThRevlist,matchlist],
          names=('Iteration','ThPrev','ThRev','ThRev=ThPrev?'))
          print (table)

          x=klist #iterations
          y=ThRevlist
          plt.plot(x,y)

          plt.xlabel ('Iterations')
          plt.ylabel('Fat Threshold')
          plt.show()
    return fatmask, fatseg, ThRev, MuscSegI
      

In [ ]:
fatseg1vol_mask_R1=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg1vol_R1=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j): 
    fatseg1vol_mask_R1[i],fatseg1vol_R1[i], ThRev, MuscSegI=optimize_threshold(roivol[i], initial_th1_R1[i])
    thresholds.append(ThRev) 
    print(f"\nSlice {i+1} Optimized Threshold #1 ={thresholds[i]}\n--------------------------------------------------------------------\n") #i+1 cus slice 1=index0, slice 15=index 14

for i in range(j):
  print (f"Slice {i+1} Optimized Threshold #1 ={thresholds[i]}")

In [ ]:
stackimages(fatseg1vol_R1)

###Threshold optimization #2

In [ ]:
#Remove the first round of fat

fig, axs = plt.subplots (j//plotx, plotx, figsize=(17,10))
roi2vol=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

def stackroi2(x=0, y=0):
  for i in range(j):
    roi2vol[i]=roivol[i]-fatseg1vol_R1[i]

    axs[y, x].imshow(roi2vol[i], cmap='bone', vmin=0, vmax=255) #don't need index here
    axs[y, x].set_title(f"slice {i+1}", fontsize=12)

    if x <(plotx-1):            
      x+=1
    else:          
      x=0
      y+=1    
  return roi2vol

stackroi2()
roi2vol=stackroi2()

In [ ]:
initial_th2_R1=[]
for i in range(j):
  initial_th2_R1.append([])
  initial_th2_R1[i]=multi_otsu(roi2vol[i])


In [ ]:
fatseg2vol_mask=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
fatseg2vol=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

thresholds2=[]
for i in range(j): 
    fatseg2vol_mask[i],fatseg2vol[i], ThRev, MuscSegI=optimize_threshold(roi2vol[i], initial_th2_R1[i])
    thresholds2.append(ThRev) #fatth will be its own list item later. fatth is just a place-holder
    print(f"\nSlice {i+1} Optimized Threshold #2 ={thresholds2[i]}\n--------------------------------------------------------------------\n") #i+1 cus slice 1=index0, slice 15=index 14


for i in range(j):
  print(f"Slice {i+1} Optimized Threshold #2 ={thresholds2[i]}")


In [ ]:
#apply Th2 to get fat from Th1

# fatsegfinalvol_mask_R1
# fatsegfinalvol_R1

fatsegfinalvol_mask_R1=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
fatsegfinalvol_R1=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
for i in range(j):
    fatsegfinalvol_mask_R1[i] = label(roivol[i]>thresholds2[i])
    fatsegfinalvol_mask_R1[i] = np.uint8(fatsegfinalvol_mask_R1[i]) 
    fatsegfinalvol_mask_R1[i] = (morphology.remove_small_objects(fatsegfinalvol_mask_R1[i],min_size=8, connectivity=1)).astype(int)
    ret, fatsegfinalvol_mask_R1[i] = cv2.threshold(fatsegfinalvol_mask_R1[i],0,1,cv2.THRESH_BINARY) 
   
    fatsegfinalvol_R1[i]= fatsegfinalvol_mask_R1[i]*roivol[i]   
    print (f"Slice {i+1} Area= {np.sum(fatsegfinalvol_R1[i]>0)}")
stackimages(fatsegfinalvol_R1)


In [ ]:
fatseg2_refinedvol_mask= (fatsegfinalvol_mask_R1 != fatseg1vol_mask_R1) #finer fat from Th2 loop not captured in Th1 loop


fatseg2_refinedvol=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8') #added!
for i in range(j):
    fatseg2_refinedvol[i]=fatseg2_refinedvol_mask[i]*roivol[i]
     

stackimages(fatseg2_refinedvol)


##Round 2: Fat Segmention + Z-Connectivity Check

In [ ]:
zcheck1vol=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
zcheck_Th2R1=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

overlay1=[]
overlay2=[]

for i in range(j):
    overlay1.append([])
    overlay2.append([])
    
    zcheck1vol[i]=(roivol[i]>thresholds[i]).astype(int)
    zcheck_Th2R1[i]=(roivol[i]>thresholds2[i]).astype(int)
      
    overlay1[i]= np.ma.masked_where(zcheck1vol[i] == 0, zcheck1vol[i])
    overlay2[i]= np.ma.masked_where(zcheck_Th2R1[i] == 0, zcheck_Th2R1[i])  
    
num=7

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(roivol[num], cmap='bone')
axs[1].imshow(roivol[num], cmap='bone')
axs[1].imshow(overlay2[num], cmap="bwr", vmin=0, vmax=1)
axs[1].imshow(overlay1[num], cmap="autumn", vmin=0, vmax=1) 

####Single Slice

In [ ]:
#Test for one slice
#1. Get Z connections

num=0

fatcombined_prev=zcheck_Th2R1[num-1]+zcheck_Th2R1[num]
fatcombined_next=zcheck_Th2R1[num]+zcheck_Th2R1[num+1]
ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

fatconnected_either=fatconnectedparts_prev+fatconnectedparts_next
ret, fatconnectedparts_either_mask= cv2.threshold(fatconnected_either,0,1,cv2.THRESH_BINARY)


fig, axs=plt.subplots (1,6,figsize=(20,20))
axs[0].imshow(zcheck_Th2R1[num-1])
axs[0].set_title(f"slice 7 raw threshold #2", fontsize=12)
axs[1].imshow(zcheck_Th2R1[num],cmap="bone")
axs[1].set_title("slice 8 raw threshold #1", fontsize=12) 
axs[2].imshow(zcheck_Th2R1[num+1])
axs[2].set_title("slice 9 raw threshold #2", fontsize=12) 
axs[3].imshow(fatconnectedparts_prev)
axs[3].set_title("z-connection-prev #1", fontsize=12) 
axs[4].imshow(fatconnectedparts_next)
axs[4].set_title("z-connection-next #1", fontsize=12) 
axs[5].imshow(fatconnectedparts_either_mask)
axs[5].set_title("combine next and prev", fontsize=12) 


In [ ]:
#2. Put Z-connection coordinates into list 
coordinates= np.argwhere(fatconnectedparts_either_mask ==1)

In [ ]:
#this is separate from the above.
#need to establish teh Z-connection for every iteration, instead of just pulling from the thresholds[i]
#OG
def optimize_threshold1_R2(i,roivar,zcheck):
    pixels=[]

    for a in range(len(roivar)):
        for b in range(len(roivar[a])):
            if roivar[a,b]!=0:          
                pixels.append(roivar[a,b])
    pixels=np.array(pixels)
    pixels=np.uint8(pixels)

    otsuth2_R2, fatots = cv2.threshold(pixels,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)   
    
    print ("Otsu Threshold =",otsuth2_R2)
    
    
    k=1
    ThPrev_R2=0
    ThRev_R2=otsuth2_R2
    while ThRev_R2!=ThPrev_R2:
        ThPrev_R2=ThRev_R2
        prefatmask_R2 = label(roivar>ThRev_R2)
        fatmask_R2 = np.uint8(prefatmask_R2)
        ret, fatmask_R2 = cv2.threshold(fatmask_R2,0,1,cv2.THRESH_BINARY)
        #NEW: find Z-connection
        fatcombined_prev=zcheck[i-1]+fatmask_R2 #note can't use zcheck[i] cus ThRev changes with each iteration
        fatcombined_next=fatmask_R2+zcheck[i+1]
        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_prev+fatconnectedparts_next
        ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list
        im_ff=fatmask_R2.copy()
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)
        #Remove small islands for Non-Z parts
        nonZ =(im_ff==1)
        Z = (im_ff==2)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        prefatseg1_R2=Z+nonZ_keep #change name??
        fatseg1_R2=prefatseg1_R2*Ivol8c_roi[i] #don't need to make a mask first
        #Fat and Muscle Quantification
        preMuscSegM_R2=roivar-fatseg1_R2
        MuscSegM_R2=np.ma.masked_where(preMuscSegM_R2 == 0, preMuscSegM_R2)
        FatSegM_R2=np.ma.masked_where(fatseg1_R2==0,fatseg1_R2)
        MuscSegI_R2=np.mean(MuscSegM_R2)
        FatSegI_R2=np.mean(FatSegM_R2)
        ThRev_R2=(1+((FatSegI_R2-MuscSegI_R2)/FatSegI_R2))*MuscSegI_R2
        def check(im,to_overlay,x):
            overlay = np.ma.masked_where(to_overlay == 0, roi)
            axs[x].imshow(im, cmap="bone")
            axs[x].imshow(overlay, cmap="hsv", vmin=0, vmax=1)
        print (f"Iteration={k}\n\tThPrev={ThPrev_R2}\n\tThRev={ThRev_R2}")
        # fig, axs = plt.subplots (1,7, figsize=(27,20))
        # axs[0].imshow(zcheck[i-1], cmap='bone')
        # axs[0].set_title(f"Slice {i} R1")
        # axs[1].imshow(fatmask_R2,cmap="bone")
        # axs[1].set_title(f"Slice {i+1} raw threshold")
        # axs[2].imshow(zcheck[i+1],cmap="bone")
        # axs[2].set_title(f"Slice {i+2} R1")
        # axs[3].imshow(z_connection,cmap="bone")
        # axs[3].set_title(f"Slice{i},{i+1},{i+2} Z-connections")
        # check(fatmask_R2, z_connection,4)
        # axs[4].set_title(f"connected_on_soi")
        # axs[5].imshow(im_ff)
        # axs[5].set_title(f"Slice {i+1} SOI FF")
        # axs[6].imshow(prefatseg1_R2)
        # axs[6].set_title(f"Slice {i+1} R2 Final Mask")
        fig, axs = plt.subplots (1,6, figsize=(15,15))
        axs[0].imshow(im_ff)
        axs[0].set_title(f"Slice {i+1} SOI FF")
        axs[1].imshow(Z,cmap="bone")
        axs[1].set_title(f"Slice {i+1} Z")
        axs[2].imshow(nonZ,cmap="bone")
        axs[2].set_title(f"Slice {i+1} nonZ")
        axs[3].imshow(nonZ_keep,cmap="bone")
        axs[3].set_title(f"Slice {i+1} nonZ_keep")
        axs[4].imshow(prefatseg1_R2,cmap="bone")
        axs[4].set_title(f"Slice {i+1} R2 Final Mask")
        axs[5].imshow(fatseg1_R2,cmap="bone")
        axs[5].set_title(f"Slice {i+1} R2 Final ")
        k+=1
        if k==50:
            break
    thresholds_R2=ThRev_R2
    return prefatseg1_R2,fatseg1_R2, thresholds_R2
prefatseg1_R2,fatseg1_R2,thresholds_R2=optimize_threshold1_R2(7,roivol[7],zcheck_Th2R1)


print (np.sum(fatseg1_R2>0))


In [ ]:
#Th2 R2

i=7
roi2_R2=roivol[i]-fatseg1_R2 

prefatseg2_R2,fatseg2_R2, thresholds2_R2=optimize_threshold1_R2(7,roi2_R2,zcheck_Th2R1)



In [ ]:
fig, ax= plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(fatseg1vol_R1[7],cmap='bone')
#ax[0].set_title(f"Slice 8; no 3D; Th= {thresholds[7]} ")
ax[1].imshow(fatseg1_R2, cmap='bone')
#ax[1].set_title(f"Slice 8; 3D connected; Th= {thresholds_R2} ")


####Volume

#####Threshold Optimization #1

In [ ]:
# initial threshold 
initial_th1_R2=[]

for i in range(j):
  initial_th1_R2.append([])
  initial_th1_R2[i]=multi_otsu(roivol[i])

In [ ]:
def optimize_thresholdvol_R2(i, roivar, ots, zcheck):
     
    k=1
    ThPrev_R2=0 
    ThRev_R2= ots[i] 
    x=0
    y=0
    while ThRev_R2!=ThPrev_R2:
        ThPrev_R2=ThRev_R2

        prefatmask_R2 = (roivar[i]>ThRev_R2)
        prefatmask_R2 = np.uint8(prefatmask_R2)
        ret, fatmask_R2 = cv2.threshold(prefatmask_R2,0,1,cv2.THRESH_BINARY)  #need this??

        if i==0:
            fatcombined_next=zcheck[i+1]+fatmask_R2
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
   
        elif i==(j-1):
            fatcombined_prev=zcheck[i-1]+fatmask_R2
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev
   
        else:
            fatcombined_prev=zcheck[i-1]+fatmask_R2
            fatcombined_next=fatmask_R2+zcheck[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY) 
        
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
        im_ff=fatmask_R2.copy()
        h, w = im_ff.shape[:2] #added
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)
        
        #Remove small islands for Non-Z parts
        nonZ =label(im_ff==1)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)

        Z = (im_ff==2)
  
        prefatseg1_R2=(Z+nonZ_keep) #do we need int cus boolean??
        fatseg1_R2=prefatseg1_R2*Ivol8c_roi[i]
        #Fat and Muscle Quantification
        preMuscSegP_R2=roivol[i]-fatseg1_R2
        MuscSegP_R2=np.ma.masked_where(preMuscSegP_R2 == 0, preMuscSegP_R2)
        FatSegP_R2=np.ma.masked_where(fatseg1_R2==0,fatseg1_R2) 
        MuscSegI_R2=np.mean(MuscSegP_R2)
        FatSegI_R2=np.mean(FatSegP_R2)
        ThRev_R2=(1+((FatSegI_R2-MuscSegI_R2)/FatSegI_R2))*MuscSegI_R2 
        print (f"Slice {i+1} Iteration={k}\n\tThPrev={ThPrev_R2}\n\tThRev={ThRev_R2}\n")

        k+=1
        if k==50:
            break
    #  if i==8:
    #       fig, axs = plt.subplots (1,9, figsize=(15,10))
    #       axs[0].imshow(zcheck[i-1],cmap="bone")
    #       axs[0].set_title(f"Slice {i} R1")     
    #       axs[1].imshow(fatmask_R2,cmap="bone")
    #       axs[1].set_title(f"Slice {i+1} raw threshold")
    #       axs[2].imshow(zcheck[i+1],cmap="bone")
    #       axs[2].set_title(f"Slice {i+2} raw threshold")

    #       axs[3].imshow(im_ff)
    #       axs[3].set_title(f"Slice {i+1} SOI FF")
    #       axs[4].imshow(Z,cmap="bone")
    #       axs[4].set_title(f"Slice {i+1} Z")
    #       axs[5].imshow(nonZ,cmap="bone")
    #       axs[5].set_title(f"Slice {i+1} nonZ")
    #       axs[6].imshow(nonZ_keep,cmap="bone")
    #       axs[6].set_title(f"Slice {i+1} nonZ_keep")
    #       axs[7].imshow(prefatseg1_R2,cmap="bone")
    #       axs[7].set_title(f"Slice {i+1} R2 Final Mask")
    #       axs[8].imshow(fatseg1_R2,cmap="bone")
    #       axs[8].set_title(f"Slice {i+1} R2 Final ")
    # fig, axs = plt.subplots (1,7, figsize=(27,20))  
    # if i>0:  
    #     axs[0].imshow(zcheck[i-1],cmap="bone")
    # else:
    #     axs[0].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")
    # axs[0].set_title(f"Slice {i} R1")     
    # axs[1].imshow(fatmask_R2,cmap="bone")
    # axs[1].set_title(f"Slice {i+1} raw threshold")
    
    # if i<(j-1):  
    #     axs[2].imshow(zcheck[i+1],cmap="bone")
    # else:
    #     axs[2].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")           
    # axs[2].set_title(f"Slice {i+2} R1")
    # axs[3].imshow(im_ff)
    # axs[3].set_title(f"Slice {i+1} SOI FF")
    # axs[4].imshow(prefatseg1_R2,cmap="bone")
    # axs[4].set_title(f"Slice {i+1} R2 Final Mask")
    # axs[5].imshow(fatseg1_R2,cmap="bone")
    # axs[5].set_title(f"Slice {i+1} R2 Final ")
    # axs[6].imshow(fatsegfinalvol_mask_R1[i],cmap="bone")
    # axs[6].set_title(f"Slice {i+1} R1 Final ")
    thresholds_R2=ThRev_R2
    return prefatseg1_R2, fatseg1_R2, thresholds_R2

In [ ]:
#Original + Original Revised
#Th1 R2
fatseg1vol_mask_R2_OG_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg1vol_R2_OG_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') #change name

thresholds_R2_OG_rev=[] #change name compared to single slice?
for i in range(j):
    thresholds_R2_OG_rev.append([])
    fatseg1vol_mask_R2_OG_rev[i],fatseg1vol_R2_OG_rev[i], thresholds_R2_OG_rev[i]=optimize_thresholdvol_R2(i, roivol,initial_th1_R2,fatsegfinalvol_mask_R1)  #fatsegfinalvol_mask_R1=round 1 final fat mask with islands REMOVED
    print (f"Slice {i+1} Th1 R2 = {thresholds_R2_OG_rev[i]}")
    print ("-----------------------------------------------------------")

for i in range(j):
    print (f"Slice {i+1} Th1 R2 OG/Rev={thresholds_R2_OG_rev[i]}")

In [ ]:
#Kenneth's
#Th1 R2
fatseg1vol_mask_R2_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg1vol_R2_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') #change name

thresholds_R2_ken=[] 
for i in range(j):
    thresholds_R2_ken.append([])
    fatseg1vol_mask_R2_ken[i],fatseg1vol_R2_ken[i], thresholds_R2_ken[i]=optimize_thresholdvol_R2(i, roivol, initial_th1_R2, zcheck_Th2R1)  #zcheck_Th2R1= round 1 final fat islands NOT removed
    print (f"Slice {i+1} Th1 R2 = {thresholds_R2_ken[i]}")
    print ("-----------------------------------------------------------")

for i in range(j):
    print (f"Slice {i+1} Th1 R2 Ken={thresholds_R2_ken[i]}")

#####Threshold Optimization #2

In [ ]:
#Original + Original Revised
roi2vol_R2_OG_rev=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') #change name
roi2vol_R2_OG_rev=roivol-fatseg1vol_R2_OG_rev

stackimages(roi2vol_R2_OG_rev)

In [ ]:
#Kenneth's

roi2vol_R2_ken=roivol-fatseg1vol_R2_ken

stackimages(roi2vol_R2_ken)

In [ ]:
#Original + Original Revised
initial_th2_R2_OGrev=[]
for i in range(j):
  initial_th2_R2_OGrev.append([])
  initial_th2_R2_OGrev[i]=multi_otsu(roi2vol_R2_OG_rev[i])


In [ ]:

#Th2 R2
fatseg2vol_mask_R2_OG_rev=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 
fatseg2vol_R2_OG_rev=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 

thresholds2_R2_OG_rev=[]
for i in range(j):
    thresholds2_R2_OG_rev.append([])
    fatseg2vol_R2_OG_rev[i],fatseg2vol_mask_R2_OG_rev[i], thresholds2_R2_OG_rev[i]=optimize_thresholdvol_R2(i, roi2vol_R2_OG_rev, initial_th2_R2_OGrev, fatsegfinalvol_mask_R1) 
    print (f"Slice {i+1} Th2 R2 = {thresholds2_R2_OG_rev[i]}")
    print ("--------------------------------------------------")
for i in range(j):
    print (f"Slice {i+1} Th2 R2 OG/Rev={thresholds2_R2_OG_rev[i]}")

In [ ]:
#Kenneth's 
initial_th2_R2_ken=[]
for i in range(j):
  initial_th2_R2_ken.append([])
  initial_th2_R2_ken[i]=multi_otsu(roi2vol_R2_ken[i])

In [ ]:
#Th2 R2
fatseg2vol_mask_R2_ken=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 
fatseg2vol_R2_ken=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') 

thresholds2_R2_ken=[]
for i in range(j):
    thresholds2_R2_ken.append([])
    fatseg2vol_R2_ken[i],fatseg2vol_mask_R2_ken[i], thresholds2_R2_ken[i]=optimize_thresholdvol_R2(i, roi2vol_R2_ken, initial_th2_R2_ken, zcheck_Th2R1) 
    print (f"Slice {i+1} Th2 R2 = {thresholds2_R2_ken[i]}")

for i in range(j):
    print (f"Slice {i+1} Th2 R2 Ken={thresholds2_R2_ken[i]}")

##Round 3: Final Z-Connectivity Check

In [ ]:
#Original Revised

zcheck_Th2R2_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    zcheck_Th2R2_rev[i] = (roivol[i]>thresholds2_R2_OG_rev[i]).astype(int) 
plt.imshow(zcheck_Th2R2_rev[7],cmap="bone")

In [ ]:
#Kenneth's

zcheck_Th2R2_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    zcheck_Th2R2_ken[i] = (roivol[i]>thresholds2_R2_ken[i]).astype(int) 
plt.imshow(zcheck_Th2R2_ken[7],cmap="bone")

In [ ]:
def R3_zcheck(threshold,zcheck):
    prefatmask_R3 = (roivol[i]>threshold[i])
    fatmask_R3 = np.uint8(prefatmask_R3)

    if i==0:
        fatcombined_next=zcheck[i+1]+fatmask_R3
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_next      
    elif i==(j-1):
        fatcombined_prev=zcheck[i-1]+fatmask_R3  
        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_prev    
    else:
        fatcombined_prev=zcheck[i-1]+fatmask_R3
        fatcombined_next=zcheck[i+1]+fatmask_R3 

        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

        z_connection=fatconnectedparts_prev+fatconnectedparts_next
        ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)      

        
    coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
    im_ff=fatmask_R3.copy()
    h, w = im_ff.shape[:2] 
    mask = np.zeros((h+2, w+2), np.uint8)
    for item in range(len(coordinates)):
        cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)

    #Remove small islands for Non-Z parts
    nonZ =label(im_ff==1)
    nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
    ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)

    Z = (im_ff==2)
   
    final_fat_mask=(Z+nonZ_keep) 
    final_fat=final_fat_mask*Ivol8c_roi[i] 
    
    def check1(im,to_overlay,x):
        overlay = np.ma.masked_where(to_overlay == 0, to_overlay)
        axs[x].imshow(im, cmap="bone")
        axs[x].imshow(overlay, cmap="hsv", vmin=0, vmax=1)
    
    fig,axs=plt.subplots(1,4,figsize=(15,10))
    check1(fatmask_R3, z_connection,0)
    axs[0].set_title(f"Overlay Z-connection on Slice {i+1}")
    axs[1].imshow(nonZ,cmap="bone")
    axs[1].set_title(f"Slice {i+1} nonZ")
    axs[2].imshow(nonZ_keep,cmap="bone")
    axs[2].set_title(f"Slice {i+1} nonZ_keep")
    axs[3].imshow(final_fat_mask,cmap="bone")
    axs[3].set_title(f"Slice {i+1} final_fat_mask")
    
    print(f"Slice {i+1} final sum={np.sum(final_fat>0)}")
  
    return final_fat_mask, final_fat

In [ ]:
#Original
final_fat_mask_OG=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
final_fat_OG=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    final_fat_mask_OG[i], final_fat_OG[i]=R3_zcheck(thresholds2_R2_OG_rev,fatsegfinalvol_mask_R1) 
    final_fat_OG[i]=final_fat_mask_OG[i]*Ivol8c_roi[i]

In [ ]:
fatseg1vol_mask_R3_OG=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg2_refinedvol_mask_R3_OG=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg2_refinedvol_R3_OG=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
  fatseg1vol_mask_R3_OG[i]=final_fat_OG[i]>thresholds_R2_OG_rev[i] 
  fatseg2_refinedvol_mask_R3_OG[i]= (fatseg1vol_mask_R3_OG[i]!=final_fat_mask_OG[i])
  fatseg2_refinedvol_R3_OG[i]=fatseg2_refinedvol_mask_R3_OG[i]*Ivol8c_roi[i] 

stackimages(fatseg1vol_mask_R3_OG)
stackimages(fatseg2_refinedvol_mask_R3_OG)

In [ ]:
#Original Revised

final_fat_mask_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
final_fat_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    final_fat_mask_rev[i], final_fat_rev[i]=R3_zcheck(thresholds2_R2_OG_rev,zcheck_Th2R2_rev) 
    final_fat_rev[i]=final_fat_mask_rev[i]*Ivol8c_roi[i]


In [ ]:
fatseg1vol_mask_R3_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg2_refinedvol_mask_R3_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg2_refinedvol_R3_rev=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
  fatseg1vol_mask_R3_rev[i]=final_fat_rev[i]>thresholds_R2_OG_rev[i] 
  fatseg2_refinedvol_mask_R3_rev[i]= (fatseg1vol_mask_R3_rev[i]!=final_fat_mask_rev[i])
  fatseg2_refinedvol_R3_rev[i]=fatseg2_refinedvol_mask_R3_rev[i]*Ivol8c_roi[i] 

stackimages(fatseg1vol_mask_R3_rev)
stackimages(fatseg2_refinedvol_mask_R3_rev)

In [ ]:
#Kenneth's
final_fat_mask_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
final_fat_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    final_fat_mask_ken[i], final_fat_ken[i]=R3_zcheck(thresholds2_R2_ken,zcheck_Th2R2_ken) 
    final_fat_ken[i]=final_fat_mask_ken[i]*Ivol8c_roi[i]


In [ ]:
#Kenneth's

fatseg1vol_mask_R3_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg2_refinedvol_mask_R3_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg2_refinedvol_R3_ken=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')


for i in range(j):
  fatseg1vol_mask_R3_ken[i]=final_fat_ken[i]>thresholds_R2_ken[i]
  fatseg2_refinedvol_mask_R3_ken[i]= (fatseg1vol_mask_R3_ken[i]!=final_fat_mask_ken[i])
  fatseg2_refinedvol_R3_ken[i]=fatseg2_refinedvol_mask_R3_ken[i]*Ivol8c_roi[i] #for calculations later

stackimages(fatseg1vol_mask_R3_ken)
stackimages(fatseg2_refinedvol_mask_R3_ken)
# stackimages(fatseg2_refinedvol_R3_ken)



###Final Product

In [ ]:
for i in range(j): 
    def check2(im, roi,x):    
      overlay = np.ma.masked_where(roi == 0, roi)
      axs[x].imshow(im, cmap="bone")
      axs[x].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
      
    def fats(thigh, fat2, fat3,x):    
      overlay = np.ma.masked_where(fat2 == 0, fat2)
      overlay2= np.ma.masked_where(fat3 == 0, fat3)
      axs[x].imshow(thigh, cmap="bone")
      axs[x].imshow(overlay, cmap="autumn", vmin=0, vmax=1)
      axs[x].imshow(overlay2, cmap="bwr", vmin=0, vmax=1, alpha=1)
    
    fig, axs = plt.subplots (1, 5, figsize=(17,55))
    axs[0].set_title(f"slice {i+1}")
    axs[0].imshow(Ivol8[i], cmap='bone')
    axs[1].imshow(Ivol8c_roi[i], cmap='bone')
    check2(Ivol8c_roi[i], roimaskvol[i],2)
    fats(Ivol8c_roi[i], fatseg1vol_mask_R3_ken[i], fatseg2_refinedvol_mask_R3_ken[i],3)
    axs[3].set_title("with Z connectivity")
    axs[4].imshow(final_fat_ken[i], cmap='bone')
    axs[4].set_title("with Z connectivity")
           
    


In [ ]:
#compare Z-connection check vs no check (R3 vs R1 fat)

for i in range(j): 
    fig, axs = plt.subplots (1, 5, figsize=(17,55))
    axs[0].imshow(Ivol8c_roi[i], cmap='bone')
    axs[0].set_title(f"Slice {i+1} Ivol8c_roi")

    fats(Ivol8c_roi[i], fatseg1vol_mask_R1[i], fatseg2_refinedvol_mask[i],1)
    axs[1].set_title("WITHOUT Z connections")
    axs[2].imshow(fatsegfinalvol_R1[i], cmap='bone')
    axs[2].set_title("WITHOUT Z connections")

    fats(Ivol8c_roi[i], fatseg1vol_mask_R3_ken[i], fatseg2_refinedvol_mask_R3_ken[i],3)
    axs[3].set_title("WITH Z connections")
    axs[4].imshow(final_fat_ken[i], cmap='bone')
    axs[4].set_title("WITH Z connections")

##Fat/Muscle Data

### Calculations

#####Threshold 1 + 2 Separated

In [ ]:
#test area calculation between mask vs no mask 
print(np.sum(roimaskvol[7]>0))
print(np.sum(roivol[7]>0))

fig, axs = plt.subplots (1, 2, figsize=(15,10))
axs[0].imshow(roimaskvol[7],cmap="bone")
axs[1].imshow(roivol[7],cmap="bone")

In [ ]:

muscleroi=muscleroi2=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

def calc_vol(fat):
    im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
    for i in range(j):
        muscleroi[i]=roimaskvol[i]-fat[i] 
        #MUSCLE AND FAT AREA
        MuscFatAreaPix=np.sum(roimaskvol[i]>0) #same result as roivol>0 #not musclemask>0 cus musclemask INCLUDES bone area! roimaskvol is ONLY muscle+fat. #Also don’t just do roi, need roi>0
        MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1] #multiply by x and y to convert to mm:  1 pixel here= spacing 0.9765625,0.9765625,5.0
        #FAT AREA
        FatAreaPix=np.sum(fat[i]>0) 
        FatAreaMM=FatAreaPix*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix=np.sum(muscleroi[i]>0)
        MuscAreaMM=MuscAreaPix*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc=FatAreaPix*100/MuscFatAreaPix #adding 2 dashes rounds it DOWN to whole number
        #VOLUME OF EACH SLICE
        MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
        FatVolMM=FatAreaMM*im_spacing[2] #multiply by z -slice thickness
        MuscVolMM=MuscAreaMM*im_spacing[2]
        print(f"\nSlice {i+1}\n")
        print(tabulate([['Musc + Fat Area', "%.1f" % MuscFatAreaPix,'pixels'], ['Musc + Fat Area',"%.1f" % MuscFatAreaMM,'mm^2'],
                ['Fat Area', "%.1f" % FatAreaPix,'pixels'],  ['Fat Area', "%.1f" % FatAreaMM,'mm^2'], ['Musc Area',"%.1f" % MuscAreaPix,'pixels'],['Musc Area',"%.1f" % MuscAreaMM,'mm^2'],
                ['Fat Perc', "%.1f" % FatPerc,'%'],['Musc + Fat Vol', "%.1f" % MuscFatVolMM,'mm^3'],['Fat Vol', "%.1f" % FatVolMM,'mm^3'],['Musc Vol', "%.1f" % MuscVolMM,'mm^3']],
                headers=['Data', 'Value','Units']))
        print ("------------------------------------------------------------")

#calc_vol(fatseg1vol_R1) # R1 threshold 1 fat - no 3D check
#calc_vol(fatsegfinalvol_R1) #R1 threshold 2 fat - no 3D check
#calc_vol(final_fat) #R2 threshold 2 fat WITH final 3D check (final fat from R2)

#####Threshold 1 + 2 Combined + Fat Correction

In [ ]:

muscleroi=muscleroi2=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

#problem - printing float separately works (354.0) but not in table
def calc(fat1,fat2,fat3):
    im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
    for i in range(j):
    #MUSCLE + FAT AREA
        MuscFatAreaPix=np.sum(roivol[i]>0)   
        MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1] 
        MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
    #THRESHOLD 1 FAT
        muscleroi1=roivol[i]-fat1[i] 
        #plt.imshow(muscleroi1,cmap="bone")
        #plt.imshow(roivol[i],cmap="bone")
        #FAT AREA
        FatAreaPix1=np.sum(fat1[i]>0) 
        FatAreaMM1=FatAreaPix1*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix1=np.sum(muscleroi1>0)
        MuscAreaMM1=MuscAreaPix1*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc1=FatAreaPix1*100/MuscFatAreaPix #adding 2 dashes rounds it DOWN to whole number
        #VOLUME OF EACH SLICE
        FatVolMM1=FatAreaMM1*im_spacing[2] #multiply by z -slice thickness
        MuscVolMM1=MuscAreaMM1*im_spacing[2]
        print(f"\nSlice {i+1} Threshold #1 \n")
    
        print(tabulate([['Musc + Fat Area', "%.1f" % MuscFatAreaPix,'pixels'], ['Musc + Fat Area',"%.1f" % MuscFatAreaMM,'mm^2'],['Musc + Fat Vol', "%.1f" % MuscFatVolMM,'mm^3'],[],
                ['Musc Area 1',"%.1f" % MuscAreaPix1,'pixels'],['Musc Area 1',"%.1f" % MuscAreaMM1,'mm^2'],['Musc Vol 1', "%.1f" % MuscVolMM1,'mm^3'],[],
                ['Fat Area 1', "%.1f" % FatAreaPix1,'pixels'],  ['Fat Area 1', "%.1f" % FatAreaMM1,'mm^2'], 
                ['Fat Perc 1', "%.1f" % FatPerc1,'%'],['Fat Vol 1', "%.1f" % FatVolMM1,'mm^3']],
                headers=['Data', 'Value','Units']))
        print (f"\n")
        
    #THRESHOLD 2 FAT
        #NOT including threshold 1 fat----------------------------------------------
        muscleroi2=roivol[i]-fat2[i] 
        #FAT AREA
        FatAreaPix2=np.sum(fat2[i]>0)
        FatAreaMM2=FatAreaPix2*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix2=np.sum(muscleroi2>0)
        MuscAreaMM2=MuscAreaPix2*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc2=FatAreaPix2*100/MuscFatAreaPix
        #VOLUME OF EACH SLICE
        FatVolMM2=FatAreaMM2*im_spacing[2]
        MuscVolMM2=MuscAreaMM2*im_spacing[2]
        
      
        #FAT CORRECTION 
        Fat1masked=np.ma.masked_where(fat1[i]==0,fat1[i]) #brighter fat
        Fat2masked=np.ma.masked_where(fat2[i]==0,fat2[i]) #less bright fat
        FatSegI_1=np.mean(Fat1masked) 
        FatSegI_2=np.mean(Fat2masked)
        cfactor=(FatSegI_2/FatSegI_1) #fat correction factor
        
        
        FatVolCombined=FatVolMM1+FatVolMM2 #final volume not corrected
        FatVolMM2_C=FatVolMM2*cfactor 
        FatVolCombined_C=FatVolMM1+FatVolMM2_C #final volume corrected
        FatPerc2_C=FatPerc2*cfactor
        
        print(f"\nSlice {i+1} Threshold #2 NOT including fat 1  \n")
        print(tabulate([['Musc Area 2',"%.1f" % MuscAreaPix2,'pixels'],['Musc Area 2',"%.1f" % MuscAreaMM2,'mm^2'],['Musc Vol 2', "%.1f" % MuscVolMM2,'mm^3'],[],
                ['Fat Area 2', "%.1f" % FatAreaPix2,'pixels'],['Fat Area 2 ', "%.1f" % FatAreaMM2,'mm^2'],
                ['Fat Perc 2', "%.1f" % FatPerc2,'%'],['Fat Perc 2 Corrected', FatPerc2_C,'%'],['Fat Vol 2 Not Corrected', "%.1f" % FatVolMM2,'mm^3'],['Fat Vol 2 Corrected', "%.1f" % FatVolMM2_C,'mm^3']],
                headers=['Data', 'Value','Units']))
        print (f"\n")
        print(f"\nSlice {i+1} Fat Correction \n")
        print(tabulate([['Fat 1 Intensity', FatSegI_1,'pixels'],['Fat 2 Intensity', FatSegI_2,'mm^2'],['Fat Correction Factor', cfactor],[],],
                headers=['Data', 'Value','Units']))
        print (f"\n")
        
    
    #FINAL FAT using Threshold 2 ----------------------------------------------------
        muscleroi3=roivol[i]-fat3[i]
        #FAT AREA
        FatAreaPix3=np.sum(fat3[i]>0)
        FatAreaMM3=FatAreaPix3*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix3=np.sum(muscleroi3>0)
        MuscAreaMM3=MuscAreaPix3*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc3=FatAreaPix3*100/MuscFatAreaPix
        #VOLUME OF EACH SLICE
        FatVolMM3=FatAreaMM3*im_spacing[2] #this is same as FatVolCombined NOT corrected- use this as a check
        MuscVolMM3=MuscAreaMM3*im_spacing[2]
        
        
        print(f"\nSlice {i+1} Threshold #2 INCLUDING fat 1 \n")
        print(tabulate([['Musc Area 3',"%.1f" % MuscAreaPix3,'pixels'],['Musc Area 3',"%.1f" % MuscAreaMM3,'mm^2'],['Musc Vol 3', "%.1f" % MuscVolMM3,'mm^3'],[],
                ['Fat Area 3', "%.1f" % FatAreaPix3,'pixels'],['Fat Area 3 ', "%.1f" % FatAreaMM3,'mm^2'],
                ['Fat Perc 3', "%.1f" % FatPerc3,'%'],['Fat Vol 3', "%.1f" % FatVolMM3,'mm^3'],[],
                ['Fat Volume Combined Not Corrected', "%.1f" % FatVolCombined,'MM^3'],['Fat Volume Combined Corrected', "%.1f" % FatVolCombined_C,'MM^3']],
                headers=['Data', 'Value','Units']))
    
        
        print ("------------------------------------------------------------")
    
    
# calc(fatseg1vol_R1,fatseg2_refinedvol,fatsegfinalvol_R1)
calc(fatseg1vol_R2_ken,fatseg2_refinedvol_R3_ken,final_fat_ken)



In [ ]:
#Checking for one slice 
def fat_correction(i):
    im_spacing = image.GetSpacing() 
    fat1masked=np.ma.masked_where(fatseg1vol_R1[i]==0,fatseg1vol_R1[i])
    fat2masked=np.ma.masked_where(fatseg2_refinedvol[i]==0,fatseg2_refinedvol[i])
    FatSegI_1=np.mean(fat1masked) 
    FatSegI_2=np.mean(fat2masked)
    cfactor=(FatSegI_2/FatSegI_1)

    #FAT AREA
    FatAreaPix1=np.sum(fatseg1vol_R1[i]>0)
    FatAreaMM1=FatAreaPix1*im_spacing[0]*im_spacing[1]
    FatVolMM1=FatAreaMM1*im_spacing[2]
    
    FatAreaPix2=np.sum(fatseg2_refinedvol[i]>0)
    FatAreaMM2=FatAreaPix2*im_spacing[0]*im_spacing[1]
    FatVolMM2=FatAreaMM2*im_spacing[2]
    FatVolMM2_C=FatAreaMM2*im_spacing[2]*cfactor
    
    FatAreaPix3=np.sum(fatsegfinalvol_R1[i]>0)
    FatAreaMM3=FatAreaPix3*im_spacing[0]*im_spacing[1]

    #VOLUME OF EACH SLICE
    FatVolMM3=FatAreaMM3*im_spacing[2] #this is same as FatVolCombined NOT corrected
    FatVolMM12_notC=FatVolMM1+FatVolMM2 
    FatVolMM3_C=FatVolMM1+FatVolMM2_C #should be same as FatVolMM12_notC

    print (f"Fat 1 intensity = {np.mean(fat1masked)}\nFat 2 intensity={np.mean(fat2masked)}\nFat Correction Factor={cfactor}\nFat3 volume NOT corrected={FatVolMM3}\nFat1+2 NOT corrected={FatVolMM12_notC}\nFat corrected={FatVolMM3_C}")

fat_correction(7)

### Summary tables + Graphs

# EXPERIMENTATION

#For Siwen



In [ ]:
zcheck1vol=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
zcheck_Th2R1=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')



overlay1=[]
overlay2=[]

for i in range(j):
    overlay1.append([])
    overlay2.append([])
    
    zcheck1vol[i]=(roivol[i]>thresholds[i]).astype(int)
    zcheck_Th2R1[i]=(roivol[i]>thresholds2[i]).astype(int)
      
    overlay1[i]= np.ma.masked_where(zcheck1vol[i] == 0, zcheck1vol[i])
    overlay2[i]= np.ma.masked_where(zcheck_Th2R1[i] == 0, zcheck_Th2R1[i])  
    
num=7

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(roivol[num], cmap='bone')
axs[1].imshow(roivol[num], cmap='bone')
axs[1].imshow(overlay2[num], cmap="bwr", vmin=0, vmax=1)
axs[1].imshow(overlay1[num], cmap="autumn", vmin=0, vmax=1) 

In [ ]:
#Test for Slice 8  - one threshold 

#1. Get Z connections
fatsegotsvol=prefatmask=fatmask_R2=fatseg=preMuscSegM=fatcombined=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
prefatmask[7]= label(roivol[7]>thresholds[7]).astype(bool)
fatmask_R2[7] = cv2.normalize(src=prefatmask[7], dst=None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) 

fatcombined_prev=zcheck_Th2R1[6]+fatmask_R2[7]
fatcombined_next=fatmask_R2[7]+zcheck_Th2R1[8]
ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
fatconnected_either=fatconnectedparts_prev+fatconnectedparts_next
ret, fatconnectedparts_either= cv2.threshold(fatconnected_either,0,1,cv2.THRESH_BINARY)


fig, axs=plt.subplots (1,6,figsize=(20,20))
axs[0].imshow(zcheck_Th2R1[6])
axs[0].set_title(f"slice 7 raw threshold #2", fontsize=12)
axs[1].imshow(fatmask_R2[7],cmap="bone")
axs[1].set_title("slice 8 raw threshold #1", fontsize=12) 
axs[2].imshow(zcheck_Th2R1[8])
axs[2].set_title("slice 9 raw threshold #2", fontsize=12) 
axs[3].imshow(fatconnectedparts_prev)
axs[3].set_title("z-connection-prev #1", fontsize=12) 
axs[4].imshow(fatconnectedparts_next)
axs[4].set_title("z-connection-next #1", fontsize=12) 
axs[5].imshow(fatconnectedparts_either)
axs[5].set_title("combine next and prev", fontsize=12) 


In [ ]:
t=fatmask_R2[7]*roivol[7]

t2=roivol[7]-t
plt.imshow(t2, cmap='bone', vmax=255)

In [ ]:
#2. Put Z-connection coordinates into list 
coordinates= np.argwhere(fatconnectedparts_either ==1)

In [ ]:
#this is separate from the above.
#need to establish teh Z-connection for every iteration, instead of just pulling from the thresholds[i]

def optimize_threshold1_R2(i):
    otsuth2_R2, fatsegots_R2 = cv2.threshold(roivol[i],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
    print ("Otsu Threshold =",otsuth2_R2)
    fatth_R2 = 70 #Change this to a variable "scientific" initiatior with otsu
    k=1
    ThPrev_R2=0 
    ThRev_R2=fatth_R2 

    while ThRev_R2!=ThPrev_R2:

        ThPrev_R2=ThRev_R2
        
        prefatmask_R2 = label(roivol[i]>ThRev_R2).astype(bool).astype(int)
        prefatmask_R2 = cv2.normalize(src=prefatmask_R2, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        ret, fatmask_R2 = cv2.threshold(prefatmask_R2,0,1,cv2.THRESH_BINARY) 

        #NEW: find Z-connection
        fatcombined_prev=zcheck_Th2R1[i-1]+fatmask_R2
        fatcombined_next=fatmask_R2+zcheck_Th2R1[i+1]
        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
        
        z_connection=fatconnectedparts_prev+fatconnectedparts_next
        ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)

        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
        im_ff=fatmask_R2.copy()
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)
        
        #Remove small islands for Non-Z parts
        nonZ =(im_ff==1)
        Z = (im_ff==2)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        prefatseg1_R2=Z+nonZ_keep #added int cus was boolean
        fatseg1_R2=prefatseg1_R2*Ivol8c_roi[i] #don't need to make a mask first 
 
        #Fat and Muscle Quantification
        preMuscSegM_R2=roivol[i]-fatseg1_R2
        MuscSegM_R2=np.ma.masked_where(preMuscSegM_R2 == 0, preMuscSegM_R2)
        FatSegM_R2=np.ma.masked_where(fatseg1_R2==0,fatseg1_R2) 
        MuscSegI_R2=np.mean(MuscSegM_R2)
        FatSegI_R2=np.mean(FatSegM_R2)

        ThRev_R2=(1+((FatSegI_R2-MuscSegI_R2)/FatSegI_R2))*MuscSegI_R2 
        
        def check(im,to_overlay,x):
            overlay = np.ma.masked_where(to_overlay == 0, roi)
            axs[x].imshow(im, cmap="bone")
            axs[x].imshow(overlay, cmap="hsv", vmin=0, vmax=1)

        print (f"Iteration={k}\n\tThPrev={ThPrev_R2}\n\tThRev={ThRev_R2}")

        fig, axs = plt.subplots (1,7, figsize=(27,20))  
        axs[0].imshow(zcheck_Th2R1[i], cmap='bone') 
        axs[0].set_title(f"Slice {i} R1")     
        axs[1].imshow(fatmask_R2,cmap="bone")
        axs[1].set_title(f"Slice {i+1} raw threshold")
        axs[2].imshow(zcheck_Th2R1[i+1],cmap="bone")
        axs[2].set_title(f"Slice {i+2} R1")
        axs[3].imshow(z_connection,cmap="bone")
        axs[3].set_title(f"Slice{i},{i+1},{i+2} Z-connections")
        check(fatmask_R2, z_connection,4)
        axs[4].set_title(f"connected_on_soi")
        axs[5].imshow(im_ff)
        axs[5].set_title(f"Slice {i+1} SOI FF")
        axs[6].imshow(prefatseg1_R2)
        axs[6].set_title(f"Slice {i+1} R2 Final Mask")

        k+=1
        if k==50:
            break
    
    thresholds_R2=ThRev_R2
    return fatseg1_R2,prefatseg1_R2, thresholds_R2

fatseg1_R2,prefatseg1_R2, th_R2=optimize_threshold1_R2(7)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 12))
ax[0].imshow(roivol[7], cmap='bone')
ax[1].imshow(t, cmap='bone')

In [ ]:
t=fatmask_R2[7]*roivol[7]
plt.imshow(t, cmap='bone')

In [ ]:
fig, ax= plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(fatseg1vol_R1[7],cmap='bone')
ax[0].set_title(f"Slice 8; no 3D; Th= {thresholds[7]} ")
ax[1].imshow(fatseg1_R2, cmap='bone')
ax[1].set_title(f"Slice 8; 3D connected; Th= {th_R2} ")


In [ ]:
def optimize_threshold1_R2_vol(i, roi, zcheck):
    
    pixels=[]

    for a in range(len(roi[i])):
        for b in range(len(roi[i][a])):
            if roi[i][a,b]!=0:          
                pixels.append(roi[i][a,b])

    pixels=np.array(pixels)
    pixels=np.uint8(pixels)

    otsuth2, fatots = cv2.threshold(pixels,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    
     
    print ("Otsu Threshold =",otsuth2)
    fatth_R2 = otsuth2

    k=1
    ThPrev_R2=0 
    ThRev_R2=fatth_R2 
    x=0
    y=0
    while ThRev_R2!=ThPrev_R2:

        ThPrev_R2=ThRev_R2
        
        prefatmask_R2 = label(roi[i]>ThRev_R2).astype(bool).astype(int)
        prefatmask_R2 = cv2.normalize(src=prefatmask_R2, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        ret, fatmask_R2 = cv2.threshold(prefatmask_R2,0,1,cv2.THRESH_BINARY) 

        
        if i==0:
            fatcombined_next=zcheck[i+1]+fatmask_R2
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
   
        elif i==(j-1):
            fatcombined_prev=zcheck[i-1]+fatmask_R2
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev
   
        else:
            fatcombined_prev=zcheck_Th2R1[i-1]+fatmask_R2
            fatcombined_next=fatmask_R2+zcheck_Th2R1[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY) 
        
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
        im_ff=fatmask_R2.copy()
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)
        
        #Remove small islands for Non-Z parts
        nonZ =(im_ff==1)
        Z = (im_ff==2)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        prefatseg1_R2=Z+nonZ_keep #added int cus was boolean
        fatseg1_R2=prefatseg1_R2*Ivol8c_roi[i] #don't need to make a mask first 
 
        #Fat and Muscle Quantification
        preMuscSegM_R2=roivol[i]-fatseg1_R2
        MuscSegM_R2=np.ma.masked_where(preMuscSegM_R2 == 0, preMuscSegM_R2)
        FatSegM_R2=np.ma.masked_where(fatseg1_R2==0,fatseg1_R2) 
        MuscSegI_R2=np.mean(MuscSegM_R2)
        FatSegI_R2=np.mean(FatSegM_R2)

        ThRev_R2=(1+((FatSegI_R2-MuscSegI_R2)/FatSegI_R2))*MuscSegI_R2 

        print (f"Iteration={k}\n\tThPrev={ThPrev_R2}\n\tThRev={ThRev_R2}")

        k+=1
        if k==50:
            break
            
    fig, axs = plt.subplots (1,5, figsize=(27,20))  
   
    if i>0:  
        axs[0].imshow(zcheck_Th2R1[i-1],cmap="bone")
    else:
        axs[0].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")
        
    axs[0].set_title(f"Slice {i} R1")     
    axs[1].imshow(fatmask_R2,cmap="bone")
    axs[1].set_title(f"Slice {i+1} raw threshold")
    
    if i<(j-1):  
        axs[2].imshow(zcheck_Th2R1[i+1],cmap="bone")
    else:
        axs[2].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")
                      
    axs[2].set_title(f"Slice {i+2} R1")
    axs[3].imshow(im_ff)
    axs[3].set_title(f"Slice {i+1} SOI FF")
    axs[4].imshow(prefatseg1_R2)
    axs[4].set_title(f"Slice {i+1} R2 Final Mask")
    thresholds_R2=ThRev_R2
    return fatseg1_R2,prefatseg1_R2, thresholds_R2

In [ ]:
fatseg1vol_R2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fatseg1vol_mask_R2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

thresholds_R2=[]
for i in range(j):
    thresholds_R2.append([])
    fatseg1vol_R2[i],fatseg1vol_mask_R2[i], thresholds_R2[i]=optimize_threshold1_R2_vol(i, roivol, zcheck_Th2R1)

In [ ]:
roi2vol_R2=fatseg2vol_R2=fatseg2vol_mask_R2=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
zcheck3vol=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
zcheck4vol=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

roi2vol_R2=roivol-fatseg1vol_R2

for i in range(j):  
    zcheck3vol[i]=(roi2vol_R2[i]>thresholds[i]).astype(int)
    zcheck4vol[i]=(roi2vol_R2[i]>thresholds2[i]).astype(int)
    
thresholds2_R2=[]
for i in range(j):
    thresholds2_R2.append([])
    fatseg2vol_R2[i],fatseg2vol_mask_R2[i], thresholds2_R2[i]=optimize_threshold1_R2_vol(i, roi2vol_R2, zcheck_Th2R1)

In [ ]:
thresholds2_R2


#1. Get Z connections
fatsegotsvol=prefatmask=fatmask_R3=fatseg=preMuscSegM=fatcombined=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

zcheck5vol=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
zcheck6vol=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')


for i in range(j):
    zcheck6vol[i] = (roivol[i]>thresholds2_R2[i]).astype(int)
    
def R3_Zcheck(i):
     
    prefatmask[i] = label(roivol[i]>thresholds2_R2[i]).astype(bool)
    fatmask_R3[i] = cv2.normalize(src=prefatmask[i], dst=None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) 

    
    if i==0:
        
        fatcombined_next=zcheck6vol[i+1]+fatmask_R3[i]
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_next      
        
    elif i==(j-1):
        
        fatcombined_prev=zcheck6vol[i-1]+fatmask_R3[i]  
        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_prev    
        
    else:
    
        fatcombined_prev=zcheck6vol[i-1]+fatmask_R3[i]
        fatcombined_next=zcheck6vol[i+1]+fatmask_R3[i] 
        
        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

        z_connection=fatconnectedparts_prev+fatconnectedparts_next
        ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)      

     
    fig, axs=plt.subplots (1,4,figsize=(20,20))
    if i>0:  
        axs[0].imshow(zcheck6vol[i-1],cmap="bone")
    else:
        axs[0].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")
        
    axs[0].set_title(f"Slice {i} raw threshold", fontsize=12)
    axs[1].imshow(fatmask_R3[i],cmap="bone")
    axs[1].set_title(f"Slice {i+1} raw threshold", fontsize=12) 
    
    if i<(j-1):  
        axs[2].imshow(zcheck6vol[i+1],cmap="bone")
    else:
        axs[2].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")
        
    axs[2].set_title(f"Slice {i+2} raw threshold", fontsize=12) 
    axs[3].imshow(z_connection)
    axs[3].set_title("z connection", fontsize=12) 
    
    return z_connection



In [ ]:
z_connection_list=[]

for i in range(j):
    z_connection_list.append([])
    z_connection_list[i]=R3_Zcheck(i)

In [ ]:
#make list of z conenctions, then floodfill at those coordinates

z_connection_final=[]
im_ff_R3=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    z_connection_final.append([])
    z_connection_final[i] = np.argwhere(z_connection_list[i] ==1) #put coordinates of Z-connections into a list 
    
    im_ff_R3[i]=fatmask_R3[i].copy()
    mask = np.zeros((h+2, w+2), np.uint8)
    
    for item in range(len(z_connection_final[i])):
        cv2.floodFill(im_ff_R3[i], mask, (z_connection_final[i][item][1],z_connection_final[i][item][0]), 2)

def check_z(i):
    fig, ax = plt.subplots(1, 2, figsize = (7, 7))
    ax[0].imshow(roivol[i], cmap='bone')
    ax[1].imshow(im_ff_R3[i])

for i in range(j):
    check_z(i)

In [ ]:
#Remove small islands for Non-Z parts

nonZ_R3= np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
Z_R3 = np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
nonZ_keep_R3 =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
final_fat_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
final_fat=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    nonZ_R3[i] =(im_ff_R3[i]==1)
    Z_R3[i] = (im_ff_R3[i]==2)
    nonZ_keep_R3[i] = (morphology.remove_small_objects(nonZ_R3[i],min_size=8, connectivity=1)).astype(int)
    final_fat_mask[i]=Z_R3[i]+nonZ_keep_R3[i] #added int cus was boolean
    final_fat[i]=final_fat_mask[i]*roivol[i] #don't need to make a mask first 

plt.imshow(final_fat[7], cmap='bone')

In [ ]:
#divide final_fat into th1 and th2 components
fat_th1_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
prefat_th2_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
fat_th2_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    fat_th1_mask[i]=final_fat[i]>thresholds_R2[i]
    
    prefat_th2_mask[i]=final_fat[i]>thresholds2_R2[i]
    fat_th2_mask[i]=prefat_th2_mask[i]!=fat_th1_mask[i]
    
plt.imshow(prefat_th2_mask[7])

In [ ]:
fat_th1=fat_th1_mask*roivol
fat_th2=fat_th2_mask*roivol
fat_th=fat_th1+fat_th2

plt.imshow(fat_th[7], cmap='bone', vmax=255)